In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!pip install ortools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 21.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavi

In [4]:
from ortools.sat.python import cp_model
import uuid
import random
import math
import itertools
import sys
# Generate a random UUID
unique_id = uuid.uuid4()


In [5]:
from transformers import BertTokenizer, BertTokenizerFast
import torch
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast, BertModel
import torch.optim as optim
import json
import ast
import random
from torch import cuda
import re

import time

device = 'cuda' if cuda.is_available() else 'cpu'
print(device)


cpu


In [6]:

maxlen=50

#sentence = "Emphasize the significance of distinct values within X33 X70 X100 X16 X36 X15 X36"

class MultiTaskModel(torch.nn.Module):
    def __init__(self, num_labels_classification, num_labels_token_classification):
        super(MultiTaskModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.classifier = torch.nn.Linear(self.bert.config.hidden_size, num_labels_classification)
        self.token_classifier = torch.nn.Linear(self.bert.config.hidden_size, num_labels_token_classification)

    def forward(self, input_ids, attention_mask, task_name):
        outputs = self.bert(input_ids, attention_mask)
        sequence_output = outputs.last_hidden_state
        pooler_output = outputs.pooler_output

        if task_name == "sequence_classification":
            return self.classifier(pooler_output)
        elif task_name == "token_classification":
            return self.token_classifier(sequence_output)
        else:
            raise ValueError("Task name should be either 'sequence_classification' or 'token_classification'.")


num_labels_classification=17
num_labels_token_classification=11
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = MultiTaskModel(num_labels_classification, num_labels_token_classification)
model.load_state_dict(torch.load('/content/drive/MyDrive/model_dict_27_12_23.pth'))




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

<All keys matched successfully>

In [7]:
def eliminate_repetitions(s):
    # Regular expression pattern to find repeated sequences of 'X' followed by digits
    pattern = re.compile(r'(X\d+)\1+')

    # Replace repeated sequences with a single instance
    result = pattern.sub(r'\1', s)

    return result

# Example usage
test_strings = ["X22", "X1X1X1", "123", "X2X2", "XX33XX33"]

for string in test_strings:
    print(f"Original: {string}, Processed: {eliminate_repetitions(string)}")

Original: X22, Processed: X22
Original: X1X1X1, Processed: X1
Original: 123, Processed: 123
Original: X2X2, Processed: X2
Original: XX33XX33, Processed: XX33XX33


In [8]:

def bert_find_c(tokenizer, model, sentence, maxlen, num_labels_token_classification):
    model.eval()
    constraint= {}
    #remove ponctuation:
    sentence = re.sub(r'[^\w\s]', '', sentence)
    #print(sentence)
    inputs = tokenizer(sentence, padding='max_length', truncation=True, max_length=maxlen, return_tensors="pt")
    ids = inputs["input_ids"]
    mask = inputs["attention_mask"]

    # get relation
    task_name="sequence_classification"
    outputs = model(ids, mask, task_name)
    logits = outputs[0]
    pred = torch.argmax(logits)

    constraint['rel']=pred.item()

    # get parameters
    task_name="token_classification"
    outputs = model(ids, mask, task_name)
    logits = outputs[0]
    active_logits = logits.view(-1, num_labels_token_classification) # shape (batch_size * seq_len, num_labels)
    flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level
    tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
    token_predictions = [i for i in flattened_predictions.cpu().numpy()]
    wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)
    #print("wp_preds", wp_preds)
    word_level_predictions = []
    # we will remove cls, sep and pad
    for pair in wp_preds:
      if (pair[0] in ['[CLS]', '[SEP]', '[PAD]']):
        continue
      else:
        word_level_predictions.append(pair)

    #print("word_level_predictions", word_level_predictions)
    list_params= {pair[1] for pair in word_level_predictions if pair[1]!=0}
    #print("list params", list_params)


    for i in list_params:
      p= [pair[0] for pair in word_level_predictions if pair[1]==i]
      #print("ici p est ",p)
      param = ''.join(s.replace('##', '') for s in p)
      param=eliminate_repetitions(param)
      constraint[i]=param

    return constraint


In [9]:
"""
rel1= Relation('<=', 2, True)
#sentence = "In any comparison between X0 and X1, X0 should win"
sentence = "In this scenario, the mandate is that X0 is smaller than X22"
#sentence = "X1 is not allowed to be less than X0"
#sentence="It's non-negotiable that the absolute value of the result of X2 subtracted from X0 matches 44."
c1=Constraint([0,12],rel1)
#c8=Constraint([0,2],rel8)

constraint= bert_find_c(tokenizer, model, sentence, maxlen, num_labels_token_classification)
compareConsTargVsConstBert(c1,constraint,dict_constraints )
print(constraint)
"""

'\nrel1= Relation(\'<=\', 2, True)\n#sentence = "In any comparison between X0 and X1, X0 should win"\nsentence = "In this scenario, the mandate is that X0 is smaller than X22"\n#sentence = "X1 is not allowed to be less than X0"\n#sentence="It\'s non-negotiable that the absolute value of the result of X2 subtracted from X0 matches 44."\nc1=Constraint([0,12],rel1)\n#c8=Constraint([0,2],rel8)\n\nconstraint= bert_find_c(tokenizer, model, sentence, maxlen, num_labels_token_classification)\ncompareConsTargVsConstBert(c1,constraint,dict_constraints )\nprint(constraint)\n'

In [10]:
class Relation:
  def __init__(self, operator , arity, directed,  parameter=-999):
    self.operator = operator

    self.arity = arity
    self.directed= directed
    self.parameter= parameter

In [11]:
class Constraint:
  def __init__(self, scope, rel):
    self.id=uuid.uuid4()
    self.scope = scope
    self.rel = rel

  def is_completely_instantiated(self, example):
      list_inst_vars=list(example.keys())
      #print(self.rel.operator, "on ",self.scope," list_inst_vars: ", list_inst_vars)
      l=[1 if v in list_inst_vars else 0 for v in self.scope ]
      if(sum(l)==len(self.scope)):
        return True
      else:
        return False

  def projection(self, example):
    keys= example.keys()
    proj= [(k, example[k]) for k in self.scope if k in keys]
    return(proj)

  def is_satisfied(self, example):
    proj=self.projection(example)
    #print(" ici la projecttion est...",proj)
    vals=[t[1] for t in proj]
    my_str=''
    val=self.rel.parameter
    if self.rel.operator in['<=', '!=','<','==','>','>=']:
      x=vals[0]
      y=vals[1]
      my_str="x"+self.rel.operator+"y"
    elif self.rel.operator=='sigma_diff':
      x=vals[0]
      y=vals[1]
      z=vals[2]
      my_str="x+y != z"
    elif self.rel.operator=='||=val':
      x=vals[0]
      y=vals[1]
      my_str="abs(x-y)==val"
    elif self.rel.operator=='||!=val':
      x=vals[0]
      y=vals[1]
      my_str="abs(x-y)!=val"
    elif self.rel.operator=='||>val':
      x=vals[0]
      y=vals[1]
      my_str="abs(x-y)>val"
    elif self.rel.operator.startswith('O_val_'):
      operator=self.rel.operator.split('O_val_')[1]
      x=vals[0]
      my_str="x"+ operator + "val"
      if operator == '=':
        my_str="x == val"
    elif self.rel.operator== '||xy_zt':
      x=vals[0]
      y=vals[1]
      z=vals[2]
      t=vals[3]
      my_str=" abs(x-y) == abs(z-t)"
    elif self.rel.operator== '||!=xy_zt':
      x=vals[0]
      y=vals[1]
      z=vals[2]
      t=vals[3]
      my_str=" abs(x-y) != abs(z-t)"
    elif self.rel.operator== 'Sum':
      my_str=[ f"vals[{i}]" for i in range(len(vals))]
      oper="+"
      my_str=oper.join(my_str)
      my_str=my_str+"=="+"val"
    elif self.rel.operator== 'all_diff':
      return len(tuple(vals))==len(set(vals))
      #print("vars = ", self.scope, " vas ", vals," ? ",val)

      #print("################## my str to be evaluated is ...",my_str, "and its value is ", eval(my_str))
      #sys.exit()

    if len(my_str) !=0:

      return eval(my_str)
    else:
      print(" unhandeled relation ??????????????????????????????????")
      sys.exit()
      return None

  def check_constraint(self, example):
    if self.is_completely_instantiated(example):
      if self.is_satisfied(example):
        return True
      else:
        return False
    else:
      return True



In [12]:
class Target_net():
  def __init__(self,constraints):
    self.constraints=constraints

  def ask(self,example):
    for c in self.constraints:
      check= c.check_constraint(example)
      if not check:
        #print("Inside ask---the following constraint is violated ", c.rel.operator, " on ", c.scope)
        return False,c
    return True, "_"

In [13]:
dict_constraints={
    0: {"constraint": "all_diff", "template":"templates_all_diff", "n_params":10},
    1: {"constraint": "<", "template":"templates_param1_strictely_lt_param2", "n_params":2},
    2: {"constraint": ">", "template":"templates_param1_strictely_gt_param2", "n_params":2},
    3: {"constraint": "||!=p", "template":"abs_val_param1_minus_param_2_diff_param3", "n_params":3},
    4: {"constraint": "||xy_zt", "template":"abs_val_p1_minus_p2_equal_abs_val_p3_minus_p4", "n_params":4},
    5: {"constraint": "||=p", "template":"abs_val_p1_minus_p2_equal_p_3", "n_params":3},
    6: {"constraint": "sigma_diff", "template":"templates_sum_x1_x2_diff_x3", "n_params":3},
    7: {"constraint": "==", "template":"templates_equal_constraint", "n_params":2},
    8: {"constraint": "!=", "template":"templates_different_than", "n_params":2},
    9: {"constraint": "<=", "template":"templates_less_than", "n_params":2},
    10: {"constraint": ">=", "template":"templates_greater_than", "n_params":2},
    11: {"constraint": "||!=xy_zt", "template":"abs_val_p1_minus_p2_diff_abs_val_p3_minus_p4", "n_params":4},
    12: {"constraint": "Sum", "template":"sum_params_equal_param", "n_params":10},
    13: {"constraint": "O_val_=", "template":"templates_equal_constraint", "n_params":2},
    14: {"constraint": "O_val_!=", "template":"templates_different_than", "n_params":2},
    15: {"constraint": "||=val", "template":"abs_val_p1_minus_p2_equal_p_3", "n_params":3},
    16: {"constraint": "||!=val", "template":"abs_val_param1_minus_param_2_diff_param3", "n_params":3}
}

In [14]:
"""
nbr_vars=3
lower_bound_domain=0
upper_bound_domain=3
vars_indices= [i for i in range(nbr_vars)]


rel1= Relation('<=', 2, True)
rel2= Relation('!=',2, False)
rel3= Relation('sigma_diff',3, False )
rel4= Relation('<', 2, True )
rel5= Relation('==',2, False )
rel6= Relation('>',2, False )
rel7= Relation('>=', 2, True )
rel8=Relation('||=val',2,False,44 )
rel9=Relation('||!=val',2,False ,45)
rel16= Relation('O_val_=',1,False,41)
rel17=Relation('||xy_zt',4,False )
rel18=Relation('||!=xy_zt',4,False )
rel19= Relation('Sum',10,False,30)
rel20= Relation('all_diff',10,False)

r1= Relation('O_val_=',1,False,1 )
r9= Relation('>',2, True )
r10= Relation('<',2, True )
r19=Relation('Sum',10,False,34)
r20=Relation('all_diff',10,False)

relations=[ r9, r10]

constraints=[]

c1=Constraint([0,1],rel1)
c2=Constraint([0,2],rel2)
c3=Constraint([0,1,2],rel3)
c4=Constraint([0,2],rel4)
c5=Constraint([0,1],rel5)
c6=Constraint([0,2],rel6)

c7=Constraint([0,1],rel7)
c8=Constraint([0,2],rel8)
c9=Constraint([0,1],rel9)

c16=Constraint([0],rel16)
c17=Constraint([0,1,2,3],rel17)
c18=Constraint([0,2,3,4],rel18)
c19=Constraint([0,2,3,4],rel19)
c20=Constraint([0,2,3,4],rel20)


c3= Constraint([0, 1, 2, 3, 4],r19)
c4=Constraint([0],r1)
c5= Constraint([0, 1, 2, 3, 4],r20)


li=[c1,c2]
constraints.extend(li)

target_net= Target_net(constraints)

positif,c = target_net.ask({1: 0, 0: 1, 2: 1})

if not positif:
  print( c.scope, c.rel.operator)


cons=[c1,c2,c3,c4,c5,c6,c7,c8,c9,c16,c17,c18,c19,c20]
"""

"\nnbr_vars=3\nlower_bound_domain=0\nupper_bound_domain=3\nvars_indices= [i for i in range(nbr_vars)]\n\n\nrel1= Relation('<=', 2, True)\nrel2= Relation('!=',2, False)\nrel3= Relation('sigma_diff',3, False )\nrel4= Relation('<', 2, True )\nrel5= Relation('==',2, False )\nrel6= Relation('>',2, False )\nrel7= Relation('>=', 2, True )\nrel8=Relation('||=val',2,False,44 )\nrel9=Relation('||!=val',2,False ,45)\nrel16= Relation('O_val_=',1,False,41)\nrel17=Relation('||xy_zt',4,False )\nrel18=Relation('||!=xy_zt',4,False )\nrel19= Relation('Sum',10,False,30)\nrel20= Relation('all_diff',10,False)\n\nr1= Relation('O_val_=',1,False,1 )\nr9= Relation('>',2, True )\nr10= Relation('<',2, True )\nr19=Relation('Sum',10,False,34)\nr20=Relation('all_diff',10,False)\n\nrelations=[ r9, r10]\n\nconstraints=[]\n\nc1=Constraint([0,1],rel1)\nc2=Constraint([0,2],rel2)\nc3=Constraint([0,1,2],rel3)\nc4=Constraint([0,2],rel4)\nc5=Constraint([0,1],rel5)\nc6=Constraint([0,2],rel6)\n\nc7=Constraint([0,1],rel7)\nc8=

In [15]:

def format_template(template, **kwargs):
    try:
        formatted_template = template.format(**kwargs)
        return formatted_template
    except KeyError as e:
        print(f"Error: Missing parameter {e}")
        return None

def fill_template_sum(template, param, *params):
    params = ', '.join(map(str, params))  # Corrected: No '*' before 'params' in join
    return template.format(params=params, param=param)  # Corrected: Named placeholders in format



#I have to add data for O_val= and O_val_!= ||=val ||!=val
dict_constraints=dict(dict_constraints)
opToIndex={ dict_constraints[i]["constraint"]: i for i in range(len(dict_constraints)) }
#print(opToIndex)

def generate_sythetic_sentence_for_constraint(c):
    i={opToIndex[c.rel.operator]}
    i=i.pop()
    #print("....i",i)
    params={}
    para=-1

    if i==0 :
      params={"param"+str(k+1): f"X{c.scope[k]}" for k in range(len(c.scope))}
      selected_params = list( params.values())
      template = random.choice(templates_all_diff)
      param_template = ", ".join(selected_params)
      formatted_template=template.format(params=param_template)

    elif i==12:
      params={"param"+str(k+1): f"X{c.scope[k]}" for k in range(len(c.scope))}
      selected_params = list( params.values())
      para = c.rel.parameter
      template = random.choice(sum_params_equal_param)
      formatted_template = fill_template_sum(template, para, *selected_params)


    elif i in [13,14,15,16]:
      params={"param"+str(k+1): f"X{c.scope[k]}" for k in range(len(c.scope))}
      if c.rel.operator in["||!=val", "||=val"]:
        params["param3"]=str(c.rel.parameter)
      #print( " 2222 ||!=val ||=val ", c.scope, c.rel.operator)
      elif c.rel.operator in[ "O_val_=", "O_val_!="]:
        params["param2"]=str(c.rel.parameter)
      template = dict_constraints[i]["template"]
      template= random.choice(eval(template))
      formatted_template = format_template(template, **params)

    else:
      params={"param"+str(k+1): f"X{c.scope[k]}" for k in range(len(c.scope))}
      template = dict_constraints[i]["template"]
      template= random.choice(eval(template))
      formatted_template = format_template(template, **params)
      #print(formatted_template)

    return formatted_template,params,str(para)



In [16]:

def is_valid(s):

    # Regular expression pattern
    # ^ - start of the string
    # (X\d+|\d+) - matches either 'X' followed by one or more digits (\d+)
    # or just one or more digits
    # $ - end of the string
    pattern = re.compile(r'^(x\d+)$')
    # Check if the string matches the pattern
    return bool(pattern.match(s))

# Example usage
test_strings = ["123", "X456", "X", "12X34", "ABC", "X12X", "", "x", "x1"]

for s in test_strings:
  print( is_valid(s), s)



def compareConsTargVsConstBert(cT,cB, dict_constraints ):
  #first of all, we have to compare relations
  print(" il s'agit d'une contrainte ", cT.rel.operator)
  relT=cT.rel.operator
  relB=dict_constraints[cB['rel']]["constraint"]
  print(relT, relB)
  if relT != relB:
    return False

  # Then, we have to compare scopes
  para=-999
  scT= cT.scope
  scB= cB.values()
  scB= list(scB)
  scB=scB[1:]

  print("nous allons comprer les scopes....", scB, " ...", scT)
  #print("0000 scB....",scB)
  if cT.rel.parameter !=-999:
    para= scB[-1]
    scB=scB[:-1]
  #print("scB....",scB)
  for s in scB:
    #print("....",s)
    if not is_valid(s):
      return False
  scB = [int(s[1:]) for s in scB]
  #print( " now scB",scB)

  #now we have to compare scopes and parameters
  #first case is when the constraint has no parameter and is not directed:
  if cT.rel.operator in ["all_diff", "==", "!=", "sigma_diff"]:
    scB=set(scB)
    scT=set(scT)
    #print(scB,scT)
    scB=set(scB)
    scT=set(scT)
    if scT != scB:
      #print("they are diff")
      return False

  if cT.rel.operator == "Sum":
    scB=set(scB)
    scT=set(scT)
    #print(scB,scT)
    scB=tuple(scB)
    scT=tuple(scT)
    #print(scB,scT)
    if scT != scB or int(para) != cT.rel.parameter:
      #print("they are diff")
      return False

  elif cT.rel.operator in [ "<", ">", "<=", " >= " ]:
    scB=tuple(scB)
    scT=tuple(scT)
    #print(scB,scT)
    if scT != scB:
      #print("they are diff")
      return False
  elif cT.rel.operator in [ "||xy_zt", "||!=xy_zt" ]:
    scB=tuple(scB)
    scT=tuple(scT)
    b0,b1,b2,b3=scB
    t0,t1,t2,t3=scT
    B0=set([b0,b1])
    B1=set([b2,b3])
    T0=set([t0,t1])
    T1=set([t2,t3])
    #print(T0,B0)
    if B0 != T0 or B1!= T1:
      #print("they are diff")
      return False
  elif cT.rel.operator in ["|p1-p2|!=p3", "|p1-p2|==p3"]:
    scB=tuple(scB)
    scT=tuple(scT)
    b0,b1,b2=scB
    t0,t1,t2=scT
    B0=set([b0,b1])
    B1= b3
    T0=set([t0,t1])
    T1= t3
    if B0 != T0 or B1!= T1:
      #print("they are diff")
      return False
  elif cT.rel.operator in ["||=val", "||!=val"]:
    scB=set(scB)
    scT=set(scT)
    print("scB....",scB)
    if scB != scT:
      return False
    B1= int(para)
    T1= cT.rel.parameter
    if B1!= T1:
      return False

  elif cT.rel.operator in ["O_val_=", "O_val_!="]:
    scB=tuple(scB)
    scT=tuple(scT)
    b0  =scB
    t0  =scT
    B0=b0
    B1= int(para)
    T0=t0
    T1= cT.rel.parameter
    #print("B0,T0",B0,T0," B1,T1",B1,T1)
    if B0 != T0 or B1!= T1:
      #print("they are diff")
      return False
  return True


False 123
False X456
False X
False 12X34
False ABC
False X12X
False 
False x
True x1


In [17]:
"""
for c in cons:
  sentence,_,_ = generate_sythetic_sentence_for_constraint(c)
  print(" ")
  print("#######")
  print(c.scope," ",c.rel.operator," ", c.rel.parameter)
  print(sentence)
  constraint= bert_find_c(tokenizer, model, sentence, maxlen, num_labels_token_classification)
  print(constraint)
  print(compareConsTargVsConstBert(c,constraint,dict_constraints ))

"Ensure that X0 and X1 are not separated by exactly 45."
"""

'\nfor c in cons:\n  sentence,_,_ = generate_sythetic_sentence_for_constraint(c)\n  print(" ")\n  print("#######")\n  print(c.scope," ",c.rel.operator," ", c.rel.parameter)\n  print(sentence)\n  constraint= bert_find_c(tokenizer, model, sentence, maxlen, num_labels_token_classification)\n  print(constraint)\n  print(compareConsTargVsConstBert(c,constraint,dict_constraints ))\n\n"Ensure that X0 and X1 are not separated by exactly 45."\n'

In [18]:
"""
c9=Constraint([0,1],rel9)
c3=Constraint([0,1,2],rel3)
c9=Constraint([0,1],rel9)
c8=Constraint([0,2],rel8)
#sentence = " Verify that  X0 summed with X1 does not equal X2"
sentence ="Ensure that X0 and X1 are not separated by exactly 45."
sentence ="Make sure the separation distance between X0 and X2 equals 44."
constraint= bert_find_c(tokenizer, model, sentence, maxlen, num_labels_token_classification)
print(constraint)
compareConsTargVsConstBert(c8,constraint,dict_constraints )
"""

'\nc9=Constraint([0,1],rel9)\nc3=Constraint([0,1,2],rel3)\nc9=Constraint([0,1],rel9)\nc8=Constraint([0,2],rel8)\n#sentence = " Verify that  X0 summed with X1 does not equal X2"\nsentence ="Ensure that X0 and X1 are not separated by exactly 45."\nsentence ="Make sure the separation distance between X0 and X2 equals 44."\nconstraint= bert_find_c(tokenizer, model, sentence, maxlen, num_labels_token_classification)\nprint(constraint)\ncompareConsTargVsConstBert(c8,constraint,dict_constraints )\n'

In [19]:
###########################################################################
########################## SUDOKO PUZZLE ##################################
###########################################################################

"""
row = dict({})
for i in range(9):
  row[i]=[i*9+j for j in range(9)]
  print(row[i])

print("")
col = dict({})
for j in range(9):
  col[j]=[i*9+j for i in range(9)]
  print(col[j])

k=0
block=dict({})
for row_block in range(3):
        for col_block in range(3):
            block[k] = [(row_block * 3 + row)*9+ (col_block * 3 + col) for row in range(3) for col in range(3)]
            print(block[k])
            k=k+1

r39= Relation('all_diff',9,True )
constraints=[]

for i in range(9):
  constraints.append(Constraint(row[i],r39))
  constraints.append(Constraint(col[i],r39))
  constraints.append(Constraint(block[i],r39))

target_net= Target_net(constraints)

nbr_vars=81
lower_bound_domain=1
upper_bound_domain=9
vars_indices= [i for i in range(nbr_vars)]
fixed_vars=[]
"""


'\nrow = dict({})\nfor i in range(9):\n  row[i]=[i*9+j for j in range(9)]\n  print(row[i])\n\nprint("")\ncol = dict({})\nfor j in range(9):\n  col[j]=[i*9+j for i in range(9)]\n  print(col[j])\n\nk=0\nblock=dict({})\nfor row_block in range(3):\n        for col_block in range(3):\n            block[k] = [(row_block * 3 + row)*9+ (col_block * 3 + col) for row in range(3) for col in range(3)]\n            print(block[k])\n            k=k+1\n\nr39= Relation(\'all_diff\',9,True )\nconstraints=[]\n\nfor i in range(9):\n  constraints.append(Constraint(row[i],r39))\n  constraints.append(Constraint(col[i],r39))\n  constraints.append(Constraint(block[i],r39))\n\ntarget_net= Target_net(constraints)\n\nnbr_vars=81\nlower_bound_domain=1\nupper_bound_domain=9\nvars_indices= [i for i in range(nbr_vars)]\nfixed_vars=[]\n'

In [20]:

"""

0: 9, 1: 5, 2: 3, 3: 6, 4: 7, 5: 4, 6: 2, 7: 1, 8: 8,
9: 1, 10: 2, 11: 7, 12: 5, 13: 8, 14: 3, 15: 9, 16: 6, 17: 4,
18: 6, 19: 8, 20: 4, 21: 1, 22: 2, 23: 9, 24: 7, 25: 5, 26: 3,
27: 4, 28: 1, 29: 5, 30: 3, 31: 9, 32: 8, 33: 6, 34: 7, 35: 2,
36: 8, 37: 3, 38: 9, 39: 2, 40: 6, 41: 7, 42: 1, 43: 4, 44: 5,
45: 7, 46: 6, 47: 2, 48: 4, 49: 5, 50: 1, 51: 8, 52: 3, 53: 9,
54: 2, 55: 4, 56: 6, 57: 8, 58: 1, 59: 5, 60: 3, 61: 9, 62: 7,
63: 3, 64: 9, 65: 1, 66: 7, 67: 4, 68: 2, 69: 5, 70: 8, 71: 6,
72: 5, 73: 7, 74: 8, 75: 9, 76: 3, 77: 6, 78: 4, 79: 2, 80: 1}


[8, 17, 26, 35, 44, 53, 62, 71, 80]
[6, 7, 8, 15, 16, 17, 24, 25, 26]
[33, 34, 35, 42, 43, 44, 51, 52, 53]
[54, 55, 56, 63, 64, 65, 72, 73, 74]
[57, 58, 59, 66, 67, 68, 75, 76, 77]
[60, 61, 62, 69, 70, 71, 78, 79, 80]

"""

'\n\n0: 9, 1: 5, 2: 3, 3: 6, 4: 7, 5: 4, 6: 2, 7: 1, 8: 8,\n9: 1, 10: 2, 11: 7, 12: 5, 13: 8, 14: 3, 15: 9, 16: 6, 17: 4,\n18: 6, 19: 8, 20: 4, 21: 1, 22: 2, 23: 9, 24: 7, 25: 5, 26: 3,\n27: 4, 28: 1, 29: 5, 30: 3, 31: 9, 32: 8, 33: 6, 34: 7, 35: 2,\n36: 8, 37: 3, 38: 9, 39: 2, 40: 6, 41: 7, 42: 1, 43: 4, 44: 5,\n45: 7, 46: 6, 47: 2, 48: 4, 49: 5, 50: 1, 51: 8, 52: 3, 53: 9,\n54: 2, 55: 4, 56: 6, 57: 8, 58: 1, 59: 5, 60: 3, 61: 9, 62: 7,\n63: 3, 64: 9, 65: 1, 66: 7, 67: 4, 68: 2, 69: 5, 70: 8, 71: 6,\n72: 5, 73: 7, 74: 8, 75: 9, 76: 3, 77: 6, 78: 4, 79: 2, 80: 1}\n\n\n[8, 17, 26, 35, 44, 53, 62, 71, 80]\n[6, 7, 8, 15, 16, 17, 24, 25, 26]\n[33, 34, 35, 42, 43, 44, 51, 52, 53]\n[54, 55, 56, 63, 64, 65, 72, 73, 74]\n[57, 58, 59, 66, 67, 68, 75, 76, 77]\n[60, 61, 62, 69, 70, 71, 78, 79, 80]\n\n'

In [21]:

######################################################
################## ZEBRA PROBLEM #####################
######################################################

"""
nbr_vars=25
lower_bound_domain=0
upper_bound_domain=4
vars_indices= [i for i in range(nbr_vars)]
fixed_vars=[]

idx={
    'Nor':0, 'En': 1, 'Sp' :2, 'Uk': 3, 'Jp': 4,
    'bl':5,'Red':6,'Gre':7, 'Yel':8, 'Iv':9,
    'Cof':10,'Mil':11,'te':12,'ora':13,'wat':14,
    'Fox':15,'dog':16,'snail':17,'hors':18,'zebr':19,
    'kool':20,'parl':21, 'old':22, 'luck':23,'chest':24
}


r1= Relation('O_val_=',1,False,0 )
r2= Relation('||=val',2,False,1 )
r3= Relation('O_val_=',1,False,2 )
r4= Relation('==',2, False )
r5= Relation('>',2, True )
r6= Relation('!=',2, False )

r7= Relation('||!=val',2,False,1 )
r8= Relation('<',2, True )
r9= Relation('>=',2, True )
r10= Relation('<=',2, True )

r11= Relation('O_val_=',1,False,3 )
r12= Relation('O_val_=',1,False,4 )
r13= Relation('O_val_=',1,False,5 )

r14= Relation('O_val_<',1,True,1 )
r15= Relation('O_val_<',1,True,2 )
r16= Relation('O_val_<',1,True,3 )
r17= Relation('O_val_<',1,True,4 )
r18= Relation('O_val_<',1,True,5 )

r19= Relation('O_val_>',1,True,1 )
r20= Relation('O_val_>',1,True,2 )
r21= Relation('O_val_>',1,True,3 )
r22= Relation('O_val_>',1,True,4 )
r23= Relation('O_val_>',1,True,5 )

r24= Relation('O_val_<=',1,True,1 )
r25= Relation('O_val_<=',1,True,2 )
r26= Relation('O_val_<=',1,True,3 )
r27= Relation('O_val_<=',1,True,4 )
r28= Relation('O_val_<=',1,True,5 )

r29= Relation('O_val_>=',1,True,1 )
r30= Relation('O_val_>=',1,True,2 )
r31= Relation('O_val_>=',1,True,3 )
r32= Relation('O_val_>=',1,True,4 )
r33= Relation('O_val_>=',1,True,5 )

r34= Relation('O_val_!=',1,True,1 )
r35= Relation('O_val_!=',1,True,2 )
r36= Relation('O_val_!=',1,True,3 )
r37= Relation('O_val_!=',1,True,4 )
r38= Relation('O_val_!=',1,True,5 )

r39= Relation('all_diff',5,True )


constraints=[]


c1=Constraint([idx['Nor']],r1) # =0 Norwegian lives in the first house on the left.
c2=Constraint([idx['Nor'],idx['bl']],r2) # ||=1 The Norwegian lives next to the blue house
c3= Constraint([idx['Mil']],r3)#=2 Milk is drunk in the middle house.
c4=Constraint([idx['En'], idx['Red'] ],r4) # == The Englishwoman lives in the red house.
c5=Constraint([idx['Cof'], idx['Gre'] ],r4) # == Coffee is drunk in the green house.
c6=Constraint([idx['kool'], idx['Yel'] ],r4) # == Kools are smoked in the yellow house.
c7=Constraint([idx['Iv'],idx['Gre']],r2) # ||=1  The green house is immediately to the right of the ivory house.
c8=Constraint([idx['Gre'],idx['Iv']],r5) # > The green house is immediately to the right of the ivory house.
c9=Constraint([idx['Sp'], idx['dog'] ],r4) #==  The Spaniard owns a dog.
c10=Constraint([idx['Uk'], idx['te'] ],r4) # == The Ukrainian drinks tea.
c11=Constraint([idx['Jp'], idx['parl'] ],r4) # == The Japanese smokes Parliament.
c12=Constraint([idx['old'], idx['snail'] ],r4) # == The Oldgold smoker owns the snail.
c13=Constraint([idx['luck'], idx['ora'] ],r4) # == The Lucky Strike smoker drinks orange juice.
c14=Constraint([idx['chest'],idx['Fox']],r2) # ||=1 Chesterfield smoker lives next to the fox owner.
c15=Constraint([idx['Yel'],idx['hors']],r2) # ||=1

c16=Constraint([0,1,2,3,4],r39) # all diff
c17=Constraint([5,6,7,8,9],r39) # ||=1
c18=Constraint([10,11,12,13,14],r39) # ||=1
c19=Constraint([15,16,17,18,19],r39) # ||=1
c20=Constraint([20,21,22,23,24],r39) # ||=1



constraints=[c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,c15,c16,c17,c18,c19,c20]

target_net= Target_net(constraints)

#first_clique=[0,1,2,3,4]
#sec_clique=[5,6,7,8,9]
#th_clique=[10,11,12,13,14]
#four_clique=[15,16,17,18,19]
#fif_clique=[20,21,22,23,24]

#for i in first_clique:
#  for j in first_clique:
#    if j>i:
#      constraints.append( Constraint([i,j],r6))

#for i in sec_clique:
#  for j in sec_clique:
#    if j>i:
#      constraints.append( Constraint([i,j],r6))

#for i in th_clique:
#  for j in th_clique:
#    if j>i:
#      constraints.append( Constraint([i,j],r6))

#for i in four_clique:
#  for j in four_clique:
#    if j>i:
#      constraints.append( Constraint([i,j],r6))

#for i in fif_clique:
#  for j in fif_clique:
#    if j>i:
#      constraints.append(Constraint([i,j],r6))

"""

"\nnbr_vars=25\nlower_bound_domain=0\nupper_bound_domain=4\nvars_indices= [i for i in range(nbr_vars)]\nfixed_vars=[]\n\nidx={\n    'Nor':0, 'En': 1, 'Sp' :2, 'Uk': 3, 'Jp': 4,\n    'bl':5,'Red':6,'Gre':7, 'Yel':8, 'Iv':9,\n    'Cof':10,'Mil':11,'te':12,'ora':13,'wat':14,\n    'Fox':15,'dog':16,'snail':17,'hors':18,'zebr':19,\n    'kool':20,'parl':21, 'old':22, 'luck':23,'chest':24\n}\n\n\nr1= Relation('O_val_=',1,False,0 )\nr2= Relation('||=val',2,False,1 )\nr3= Relation('O_val_=',1,False,2 )\nr4= Relation('==',2, False )\nr5= Relation('>',2, True )\nr6= Relation('!=',2, False )\n\nr7= Relation('||!=val',2,False,1 )\nr8= Relation('<',2, True )\nr9= Relation('>=',2, True )\nr10= Relation('<=',2, True )\n\nr11= Relation('O_val_=',1,False,3 )\nr12= Relation('O_val_=',1,False,4 )\nr13= Relation('O_val_=',1,False,5 )\n\nr14= Relation('O_val_<',1,True,1 )\nr15= Relation('O_val_<',1,True,2 )\nr16= Relation('O_val_<',1,True,3 )\nr17= Relation('O_val_<',1,True,4 )\nr18= Relation('O_val_<',1,Tr

In [22]:
######################################################
################## PURDEY PROBLEM #####################
######################################################

"""
nbr_vars=12
lower_bound_domain=0
upper_bound_domain=3
vars_indices= [i for i in range(nbr_vars)]
fixed_vars=[]

idx={
    'Boyds':0, 'Garveys': 1, 'Logans' :2, 'Navarros': 3,
    'Flour': 4,'Kerosene':5,'Muslin':6,'Sugar':7,
    'Cash':8, 'Credit':9,'Traded_Ham':10,'Traded_Peas':11,
    }

# 1- The Boyds were new in town, and this was their first visit to the store
# 2- The family (which wasn't the Logan's) that traded the bushel of peas didn't buy the kerosene. ==> Logans !=Traded_Peas "and"  Traded_Peas !=Kerosene
# 3- The Boyds and the Garveys bought the kerosene and the muslin in some order
# 4- One family traded a cured ham for a large sack of flour ==> Traded_Ham = flour
# 5- Purdey only extended credit to regular customers, such that the family that bought the muslin on credit

# constraints:
# 2:  Logans !=Traded_Peas "and"  Traded_Peas !=Kerosene
# 3: (Boyds = kerosene and the Garveys = muslin) [[[[[ or (Boyds = muslin  and the Garveys = kerosene ) ]]]]] ?
# 4:  Traded_Ham = flour
# 5: Muslin = credit
# 1 and 5: Boyds !=credit




r1= Relation('<=', 2, True)
r4= Relation('==',2, False )
r6= Relation('!=',2, False )
r39= Relation('all_diff',10,True )


constraints=[]


#c1=Constraint([0,5],r4)
#c2=Constraint([0,8],r4)
#c3=Constraint([1,6],r4)

#c4=Constraint([1,9],r4)
#c5=Constraint([2,4],r4)

#c6=Constraint([2,10],r4)
#c7=Constraint([3,7],r4)

#c8=Constraint([3,11],r4)
#c9=Constraint([5,8],r4)

#c10=Constraint([6,9],r4)
#c11=Constraint([4,10],r4)
#c12=Constraint([7,11],r4)


c1=Constraint([2,11],r6) #  Logans !=Traded_Peas
c2=Constraint([11,5],r6) #  Traded_Peas !=Kerosene
c3=Constraint([4,10],r4) #Traded_Ham = flour
c4=Constraint([6,9],r4) # Muslin = credit
c5=Constraint([0,9],r6) #   Boyds !=credit
c6=Constraint([0,5],r4) #Boyds = kerosene
c7=Constraint([1,6],r4) #Garveys = muslin
c8=Constraint([0,1],r1) #Boyds < Garveys
c9=Constraint([1,2],r1) #Garveys <= Logans
c10=Constraint([2,3],r1) #Logans <= Navarros




# Formulaion Nadjib

# Families: purdey[0]=family_1 ==> Boyds purdey[1]=family_2 ==> Garveys
# purdey[2]=family_3 ==> Logans purdey[3]=family_4 ==> Navarros
# Bought: purdey[4]=bought_1 ==> Flour purdey[5]=bought_2 ==> Kerosene
# purdey[6]=bought_3 ==> Muslin purdey[7]=bought_4 ==> Sugar
# How Paid:
# purdey[8]=paid_1 ==> Cash purdey[9]=paid_2 ==> Credit
#purdey[10]=paid_3 ==> Traded Ham purdey[11]=paid_4 ==> Traded Peas

#constraints.add(new ConacqConstraint(EqualXY, purdey[0], purdey[5]));
#constraints.add(new ConacqConstraint(EqualXY, purdey[0], purdey[8]));
#constraints.add(new ConacqConstraint(EqualXY, purdey[1], purdey[6]));
#constraints.add(new ConacqConstraint(EqualXY, purdey[1], purdey[9]));
#constraints.add(new ConacqConstraint(EqualXY, purdey[2], purdey[4]));
#constraints.add(new ConacqConstraint(EqualXY, purdey[2], purdey[10]));
#constraints.add(new ConacqConstraint(EqualXY, purdey[3], purdey[7]));
#constraints.add(new ConacqConstraint(EqualXY, purdey[3], purdey[11]));
#constraints.add(new ConacqConstraint(EqualXY, purdey[5], purdey[8]));
#constraints.add(new ConacqConstraint(EqualXY, purdey[6], purdey[9]));
#constraints.add(new ConacqConstraint(EqualXY, purdey[4], purdey[10]));
#constraints.add(new ConacqConstraint(EqualXY, purdey[7], purdey[11]));



li=[c1,c2,c3,c4,c5,c6,c7,c8,c9,c10] # adding constraints for breaking symetries
#li=[c1,c2,c3,c4,c5,c6,c7]# without breaking symetries
constraints.extend(li)

first_clique=[0,1,2,3]
sec_clique=[4,5,6,7]
th_clique=[8,9,10,11]

constraints.append( Constraint(first_clique,r39))
constraints.append( Constraint(sec_clique,r39))
constraints.append( Constraint(th_clique,r39))


print("le nombre des contraintes est ", len(constraints))

target_net= Target_net(constraints)

"""


'\nnbr_vars=12\nlower_bound_domain=0\nupper_bound_domain=3\nvars_indices= [i for i in range(nbr_vars)]\nfixed_vars=[]\n\nidx={\n    \'Boyds\':0, \'Garveys\': 1, \'Logans\' :2, \'Navarros\': 3,\n    \'Flour\': 4,\'Kerosene\':5,\'Muslin\':6,\'Sugar\':7,\n    \'Cash\':8, \'Credit\':9,\'Traded_Ham\':10,\'Traded_Peas\':11,\n    }\n\n# 1- The Boyds were new in town, and this was their first visit to the store\n# 2- The family (which wasn\'t the Logan\'s) that traded the bushel of peas didn\'t buy the kerosene. ==> Logans !=Traded_Peas "and"  Traded_Peas !=Kerosene\n# 3- The Boyds and the Garveys bought the kerosene and the muslin in some order\n# 4- One family traded a cured ham for a large sack of flour ==> Traded_Ham = flour\n# 5- Purdey only extended credit to regular customers, such that the family that bought the muslin on credit\n\n# constraints:\n# 2:  Logans !=Traded_Peas "and"  Traded_Peas !=Kerosene\n# 3: (Boyds = kerosene and the Garveys = muslin) [[[[[ or (Boyds = muslin  and the

In [23]:
######################################################
################## GOLOMB PROBLEM #####################
######################################################
"""
m=8 # number of marks
nbr_vars=m
lower_bound_domain=0
upper_bound_domain=int(m*m / 2)

vars_indices= [i for i in range(nbr_vars)]

re17=Relation('||xy_zt',4,False )
re18=Relation('||!=xy_zt',4,False )


r4= Relation('==',2, False )
r5= Relation('>',2, True )
r6= Relation('!=',2, False )
r8= Relation('<',2, True )
r9= Relation('>=',2, True )
r10= Relation('<=',2, True )


constraints=[]

relations=[r4,r5,r6 , r8, r9, r10,re17,re18]
list_xy= list(  itertools.combinations(vars_indices, 2 ))
list_zt= list(  itertools.combinations(vars_indices, 2 ))
for t in list_xy:
  for u in list_zt:
    s1 = t
    s2=  u
    if s1 < s2:
      print("s1:", s1, "s2", s2)
      s=tuple([s1[0],s1[1],s2[0],s2[1]])
      constraints.append( Constraint([s1[0],s1[1],s2[0],s2[1]],re18))
      #print( '||xy_zt', '||!=xy_zt', s1[0],s1[1], " != ", s2[0],s2[1] )

print("le nombre des contraintes est ", len(constraints))


for i in range(m-1):
  constraints.append( Constraint([i,i+1],r8))


print("le nombre des contraintes est ", len(constraints))

target_net= Target_net(constraints)

"""


'\nm=8 # number of marks\nnbr_vars=m\nlower_bound_domain=0\nupper_bound_domain=int(m*m / 2)\n\nvars_indices= [i for i in range(nbr_vars)]\n\nre17=Relation(\'||xy_zt\',4,False )\nre18=Relation(\'||!=xy_zt\',4,False )\n\n\nr4= Relation(\'==\',2, False )\nr5= Relation(\'>\',2, True )\nr6= Relation(\'!=\',2, False )\nr8= Relation(\'<\',2, True )\nr9= Relation(\'>=\',2, True )\nr10= Relation(\'<=\',2, True )\n\n\nconstraints=[]\n\nrelations=[r4,r5,r6 , r8, r9, r10,re17,re18]\nlist_xy= list(  itertools.combinations(vars_indices, 2 ))\nlist_zt= list(  itertools.combinations(vars_indices, 2 ))\nfor t in list_xy:\n  for u in list_zt:\n    s1 = t\n    s2=  u\n    if s1 < s2:\n      print("s1:", s1, "s2", s2)\n      s=tuple([s1[0],s1[1],s2[0],s2[1]])\n      constraints.append( Constraint([s1[0],s1[1],s2[0],s2[1]],re18))\n      #print( \'||xy_zt\', \'||!=xy_zt\', s1[0],s1[1], " != ", s2[0],s2[1] )\n\nprint("le nombre des contraintes est ", len(constraints))\n\n\nfor i in range(m-1):\n  constraints

In [24]:
######################################################
################## KAKURU PROBLEM #####################
######################################################

grid = [
    [0, 0, 0, 0, 0, -1],
    [0, 0, 0, 0, 0, 0],
    [0, 0, -1, -1, 0, 0],
    [0, 0, -1, -1, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [-1, 0, 0, 0, 0, 0]
]


"""
horizontal_clues = [
    {34: [(0, 0), (0, 1), (0, 2),(0,3),(0,4)]}, #34
    {39: [(1, 0), (1, 1), (1, 2),(1,3),(1,4),(1,5)]},
    {4:  [(2, 0), (2, 1)]},#4
    {8:  [(2, 4), (2, 5)]},
    {7:  [(3, 0), (3, 1)]}, #7
    {9:  [(3, 4), (3, 5)]},#9
    {35: [(4, 0), (4, 1), (4, 2),(4,3),(4,4),(4,5)]},
    {15: [ (5, 1), (5, 2),(5,3),(5,4),(5,5)]}# 15
]



vertical_clues = [
    {15: [(0, 0), (1, 0), (2, 0), (3, 0),(4,0)]},
    {21: [(0, 1), (1, 1), (2, 1), (3, 1),(4,1),(5,1)]},
    {13: [(0, 2), (1, 2)]},
    {13: [(4,2),(5,2)]},
    {15: [(0, 3), (1, 3)]},
    {9:  [(4,3),(5,3)]},
    {30: [(0, 4), (1, 4), (2, 4), (3, 4),(4,4),(5,4)]},
    {35: [(1, 5), (2, 5), (3, 5),(4,5),(5,5)]}
]
"""


horizontal_clues = [
    {34: [(0, 0), (0, 1), (0, 2),(0,3),(0,4)]}, #34
    {33: [(1, 0), (1, 1), (1, 2),(1,3),(1,4),(1,5)]},
    {4:  [(2, 0), (2, 1)]},#4
    {8:  [(2, 4), (2, 5)]},
    {7:  [(3, 0), (3, 1)]}, #7
    {9:  [(3, 4), (3, 5)]},#9
    {35: [(4, 0), (4, 1), (4, 2),(4,3),(4,4),(4,5)]},
    {19: [ (5, 1), (5, 2),(5,3),(5,4),(5,5)]}# 15
]



vertical_clues = [
    {15: [(0, 0), (1, 0), (2, 0), (3, 0),(4,0)]},
    {21: [(0, 1), (1, 1), (2, 1), (3, 1),(4,1),(5,1)]},
    {13: [(0, 2), (1, 2)]},
    {13: [(4,2),(5,2)]},
    {15: [(0, 3), (1, 3)]},
    {9:  [(4,3),(5,3)]},
    {28: [(0, 4), (1, 4), (2, 4), (3, 4),(4,4),(5,4)]},
    {35: [(1, 5), (2, 5), (3, 5),(4,5),(5,5)]}
]


n=len(grid)
vars_indices=[i for i in range(n*n)]
nbr_vars= len(vars_indices)
fixed_vars=[]

lower_bound_domain=1
upper_bound_domain=9


r39= Relation('all_diff',10,True )


constraints=[]

for i in range(n):
  for j in range(n):
    if grid[i][j]==-1:
      fixed_vars.append(i*n+j)

print("printing fixed_vars",fixed_vars)


print("printing horizontal clues")
for h_c in horizontal_clues:
  clue= list(h_c.keys())[0]
  l= list(h_c.values())[0]
  list_vars = [t[0]*n+t[1] for t in  l]
  print(clue,  list_vars)
  constraints.append(Constraint(list_vars, Relation('Sum',10,False,clue)))
  constraints.append(Constraint(list_vars, r39))

print("printing vertical clues")
for v_c in vertical_clues:
  clue= list(v_c.keys())[0]
  l= list(v_c.values())[0]
  list_vars = [t[0]*n+t[1] for t in  l]
  print(clue,  list_vars)
  constraints.append(Constraint(list_vars, Relation('Sum',10,False,clue)))
  constraints.append(Constraint(list_vars, r39))


print("le nombre des contraintes est ", len(constraints))
target_net= Target_net(constraints)


printing fixed_vars [5, 14, 15, 20, 21, 30]
printing horizontal clues
34 [0, 1, 2, 3, 4]
33 [6, 7, 8, 9, 10, 11]
4 [12, 13]
8 [16, 17]
7 [18, 19]
9 [22, 23]
35 [24, 25, 26, 27, 28, 29]
19 [31, 32, 33, 34, 35]
printing vertical clues
15 [0, 6, 12, 18, 24]
21 [1, 7, 13, 19, 25, 31]
13 [2, 8]
13 [26, 32]
15 [3, 9]
9 [27, 33]
28 [4, 10, 16, 22, 28, 34]
35 [11, 17, 23, 29, 35]
le nombre des contraintes est  32


In [25]:


def add_constraint_to_cpModel(constraint,model,vars, abs_vars):
    c= constraint
    n=len(vars)
    i=j=u=w=-1
    val=c.rel.parameter
    if(c.rel.arity==1):
      i=c.scope[0]
    elif(c.rel.arity==2):
      i,j=c.scope
    elif(c.rel.arity==3):
      i,j,u=c.scope
    elif(c.rel.arity==4):
      i,j,u,w=c.scope
    if "<=" == c.rel.operator:
      model.Add(vars[i] <= vars[j])
    elif ">=" == c.rel.operator:
      model.Add(vars[i] >= vars[j])
    elif "!=" == c.rel.operator:
      model.Add(vars[i] != vars[j])
    elif "sigma_diff" == c.rel.operator:
      model.Add(vars[i] + vars[j] != vars[u])
    elif "<" == c.rel.operator:
      model.Add(vars[i] < vars[j])
    elif ">" == c.rel.operator:
      model.Add(vars[i] > vars[j])
    elif "==" == c.rel.operator:
      model.Add(vars[i] == vars[j])

    elif "||=val" == c.rel.operator:
      model.AddAbsEquality(val, vars[i] - vars[j])
    elif "||!=val" == c.rel.operator:
      #print(" len of abs_vars:  ", len(abs_vars), ",j,n, i*n+j", i, j, n, i*n+j )
      model.AddAbsEquality(abs_vars[i*n+j], vars[i] - vars[j])
      model.Add(abs_vars[i*n+j] != val)
    elif "||>val" == c.rel.operator:
      model.AddAbsEquality(abs_vars[i*n+j], vars[i] - vars[j])
      model.Add(abs_vars[i*n+j] > val)
    elif "||<val" == c.rel.operator:
      model.AddAbsEquality(abs_vars[i*n+j], vars[i] - vars[j])
      model.Add(abs_vars[i*n+j] < val)
    elif "||>=val" == c.rel.operator:
      model.AddAbsEquality(abs_vars[i*n+j], vars[i] - vars[j])
      model.Add(abs_vars[i*n+j] >= val)
    elif "||<=val" == c.rel.operator:
      model.AddAbsEquality(abs_vars[i*n+j], vars[i] - vars[j])
      model.Add(abs_vars[i*n+j] <= val)

    elif "O_val_<=" == c.rel.operator:
      model.Add(vars[i]<=val)
    elif "O_val_>=" == c.rel.operator:
      model.Add(vars[i]>=val)
    elif "O_val_<" == c.rel.operator:
      model.Add(vars[i]<val)
    elif "O_val_>" == c.rel.operator:
      model.Add(vars[i]>val)
    elif "O_val_!=" == c.rel.operator:
      model.Add(vars[i]!=val)
    elif "O_val_=" == c.rel.operator:
      model.Add(vars[i]==val)

    elif "||xy_zt" == c.rel.operator:
      model.AddAbsEquality(abs_vars[i*n+j], vars[i] - vars[j])
      model.AddAbsEquality(abs_vars[u*n+w], vars[u] - vars[w])
      model.Add(abs_vars[i*n+j] == abs_vars[u*n+w])

    elif "||!=xy_zt" == c.rel.operator:
      model.AddAbsEquality(abs_vars[i*n+j], vars[i] - vars[j])
      model.AddAbsEquality(abs_vars[u*n+w], vars[u] - vars[w])
      model.Add(abs_vars[i*n+j] != abs_vars[u*n+w])

    elif "Sum" == c.rel.operator:
      involved_vars= [vars[i] for i in c.scope]
      #print(" involved vars: ", [v.Name() for v in involved_vars])
      #sys.exit()
      model.Add(sum(involved_vars) == val)
    elif "all_diff" == c.rel.operator:
      involved_vars= [vars[i] for i in c.scope]
      model.AddAllDifferent(involved_vars)

    else:
      print(f"Unsupported constraint: {c.rel.operator}")
      sys.exit()



def generateExample(learned_tuples, learned_constraints, fixed_vars ):


    model = cp_model.CpModel()

    # Creates the variables.

    vars=[]
    for i in range(nbr_vars):
      vars.append(model.NewIntVar(lower_bound_domain, upper_bound_domain, "x_"+str(i)))


    abs_vars= [ model.NewIntVar(0, 2*upper_bound_domain, "abs_"+str(k)) for k in range(nbr_vars * nbr_vars) ]

    for i in range(nbr_vars):
      for j in range(nbr_vars):
        model.Add(abs_vars[i*nbr_vars+j] == abs_vars[j*nbr_vars+i])
        model.AddAbsEquality(abs_vars[i*nbr_vars+j], vars[i] - vars[j])

    # Creates the constraints.
    for c in learned_constraints:
          add_constraint_to_cpModel(c,model,vars,abs_vars)

    for i in fixed_vars:
      model.Add(vars[i] == 1)



    # Creates the constraints.
    for e in learned_tuples:
      forbidden_tuple = tuple(list(e.values()))
      #print(forbidden_tuple)
      inv_vars= [vars[i] for i in e.keys()]
      #print(inv_vars)
      model.AddForbiddenAssignments(inv_vars, [forbidden_tuple])


    # Creates a solver and solves the model.
    solver = cp_model.CpSolver()




    solver.parameters.randomize_search = True
    solver.parameters.random_seed = 42


    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        #print("\n".join([f"{var.Name()} = {solver.Value(var)}" for var in vars]))
        proj= {i: solver.Value(vars[i])for i in range(len(vars))}
        return proj
    else:
        #print("No solution found.")
        return({})

    proj= [solver.Value(vars[x])for x in range(nbr_vars)]
    return proj

class SolutionPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__solution_count = 0
        self.__variables = variables

    def on_solution_callback(self):
        self.__solution_count += 1
        print(f'Solution {self.__solution_count}:')
        for v in self.__variables:
            print(f'  {v.Name()} = {self.Value(v)}')

    def solution_count(self):
        return self.__solution_count

def solveTargetNetwork(constraints, fixed_vars ):

    print(fixed_vars)

    model = cp_model.CpModel()

    # Creates the variables.

    vars=[]
    for i in range(nbr_vars):
      vars.append(model.NewIntVar(lower_bound_domain, upper_bound_domain, "x_"+str(i)))


    abs_vars= [ model.NewIntVar(0, 2*upper_bound_domain, "abs_"+str(k)) for k in range(nbr_vars * nbr_vars) ]


    for i in range(nbr_vars):
      for j in range(nbr_vars):
        model.Add(abs_vars[i*nbr_vars+j] == abs_vars[j*nbr_vars+i])
        model.AddAbsEquality(abs_vars[i*nbr_vars+j], vars[i] - vars[j])

    # Creates the constraints.
    for c in constraints:
      print(" adding ",c.rel.operator, " on ", c.scope)
      add_constraint_to_cpModel(c,model,vars,abs_vars)

    for i in fixed_vars:
      model.Add(vars[i] == 1)

    # Creates a solver and solves the model.
    solver = cp_model.CpSolver()
    solution_printer = SolutionPrinter(vars)
    status = solver.SearchForAllSolutions(model, solution_printer)




In [26]:
solveTargetNetwork(constraints, fixed_vars)

[5, 14, 15, 20, 21, 30]
 adding  Sum  on  [0, 1, 2, 3, 4]
 adding  all_diff  on  [0, 1, 2, 3, 4]
 adding  Sum  on  [6, 7, 8, 9, 10, 11]
 adding  all_diff  on  [6, 7, 8, 9, 10, 11]
 adding  Sum  on  [12, 13]
 adding  all_diff  on  [12, 13]
 adding  Sum  on  [16, 17]
 adding  all_diff  on  [16, 17]
 adding  Sum  on  [18, 19]
 adding  all_diff  on  [18, 19]
 adding  Sum  on  [22, 23]
 adding  all_diff  on  [22, 23]
 adding  Sum  on  [24, 25, 26, 27, 28, 29]
 adding  all_diff  on  [24, 25, 26, 27, 28, 29]
 adding  Sum  on  [31, 32, 33, 34, 35]
 adding  all_diff  on  [31, 32, 33, 34, 35]
 adding  Sum  on  [0, 6, 12, 18, 24]
 adding  all_diff  on  [0, 6, 12, 18, 24]
 adding  Sum  on  [1, 7, 13, 19, 25, 31]
 adding  all_diff  on  [1, 7, 13, 19, 25, 31]
 adding  Sum  on  [2, 8]
 adding  all_diff  on  [2, 8]
 adding  Sum  on  [26, 32]
 adding  all_diff  on  [26, 32]
 adding  Sum  on  [3, 9]
 adding  all_diff  on  [3, 9]
 adding  Sum  on  [27, 33]
 adding  all_diff  on  [27, 33]
 adding  Sum  on

In [27]:
"""
e={1: 0, 0: 1, 2: 1}
e2={0: 0, 1: 1, 2: 1}
e3={0: 0, 1: 0, 2: 0}
e4={0: 0, 1: 1, 2: 0}
e5={0: 1, 1: 1, 2: 1}
e6={0: 0, 1: 0, 2: 1}
e7={0: 1, 1: 0, 2: 0}
e8={0: 1, 1: 1, 2: 0}
L=[e,e2,e3,e4,e5,e6,e7,e8]
generateExample(L)
"""

'\ne={1: 0, 0: 1, 2: 1}\ne2={0: 0, 1: 1, 2: 1}\ne3={0: 0, 1: 0, 2: 0}\ne4={0: 0, 1: 1, 2: 0}\ne5={0: 1, 1: 1, 2: 1}\ne6={0: 0, 1: 0, 2: 1}\ne7={0: 1, 1: 0, 2: 0}\ne8={0: 1, 1: 1, 2: 0}\nL=[e,e2,e3,e4,e5,e6,e7,e8]\ngenerateExample(L)\n'

In [28]:
##### some usefull functions.

def exps_union(e1,e2):
  v_1= list(e1.keys())
  #print("#",v_1)
  v_2= list(e2.keys())
  v= sorted(set(v_1+v_2))
  merged_e = {k: e1[k] for k in v if k in v_1}
  merged_e.update({k: e2[k] for k in v if k not in v_1})
  return merged_e

def split_exple(e):
  li= list(e.items())
  size=len(li)
  size_left = math.ceil(size/2)
  left={}
  right={}
  for k in range(size_left):
    key, value = li[k]
    left[key]=value
  for k in range(size_left,size):
    key, value = li[k]
    right[key]=value
  return left,right

def difference(l1,l2):
  difference = list(set(l1) - set(l2))
  return difference


In [29]:
def findScope(R,Y, Learned_constraints):
  global cpt
  e_R=R
  cpt =cpt+1
  #####################################################################
  answer,cT= target_net.ask(e_R)
  #### cpt,AskRec_UniAcq,AskRec_FindTuple,Ask_FindTuple,Ask_UniAcq,answer
  file.write(f"{cpt},,,1,,{answer}\n")
  ###########################################################################

  if not answer:
      print(" nous sommes ds find tuple, et c'est un exemple negatif. Pouvez donner une seule raison? ")
      sentence,_,_ = generate_sythetic_sentence_for_constraint(cT)
      print(sentence)
      cB= bert_find_c(tokenizer, model, sentence, maxlen, num_labels_token_classification)
      cpt=cpt+1
      ###############################################################################
      comp=compareConsTargVsConstBert(cT,cB,dict_constraints )
      print(" bert a réussi ?", comp)
      #### cpt,AskRec_UniAcq,AskRec_FindTuple,Ask_FindTuple,Ask_UniAcq,answer
      file.write(f"{cpt},,1,,,{comp}\n")
      #############################################################################
      if comp:
        Learned_constraints.append(cT)
        return "Exit"
      else:
        return {}

  if len(Y)==1:
    return Y


  Y1,Y2 = split_exple(Y)
  R_union_Y1= exps_union(R,Y1)
  S1=findScope(R_union_Y1,Y2, Learned_constraints)
  if S1=="Exit":
    return "Exit"
  R_union_S1=exps_union(R,S1)
  S2={}
  S2=findScope(R_union_S1,Y1, Learned_constraints)
  if S2=="Exit":
    return "Exit"
  return exps_union(S1,S2)

In [30]:

"""
e={0: 1, 1: 0, 2: 2, 3: 0, 4: 0}

print("I m here e:",e, "result of find scope..", findScope({},e))
"""

'\ne={0: 1, 1: 0, 2: 2, 3: 0, 4: 0}\n\nprint("I m here e:",e, "result of find scope..", findScope({},e))\n'

In [31]:
#Image('algo.png',width=550, height=400)


In [32]:

def fineGrainedAcq():
  global cpt
  cutoff=0
  L=[]
  sols=[]
  L_union_sols=[]
  Learned_constraints=[]
  convergence = False
  while not convergence:
    print(" cpt ",cpt)
    if cutoff>10:
      return L, Learned_constraints

    L_union_sols=L+sols
    e=generateExample(L_union_sols, Learned_constraints,fixed_vars)
    if(len(e)==0):
      print("convergence......")
      return L, Learned_constraints
    cpt=cpt+1
    ########################################################################
    answer,cT = target_net.ask(e)
    # cpt,AskRec_UniAcq,AskRec_FindTuple,Ask_FindTuple,Ask_UniAcq,answer
    file.write(f"{cpt},,,,1,{answer}\n")
    #########################################################################
    if answer:
      sols.append(e)
      cutoff=cutoff+1
      print(" sol ", e)
    else:
      cutoff=0
      sentence,_,_ = generate_sythetic_sentence_for_constraint(cT)
      print("c'est un exemple negatif, pouvez expliquer pourquoi?")
      print(sentence)
      cB= bert_find_c(tokenizer, model, sentence, maxlen, num_labels_token_classification)
      cpt=cpt+1
      ##########################################################################
      comp=compareConsTargVsConstBert(cT,cB,dict_constraints )
      print(" bert a réussi ? ", comp)
      ####### cpt,AskRec_UniAcq,AskRec_FindTuple,Ask_FindTuple,Ask_UniAcq,answer
      file.write(f"{cpt},1,,,,{comp}\n")
      ##########################################################################
      if comp:
        Learned_constraints.append(cT)
      else:
        forbiddenTuple=findScope({},e, Learned_constraints)
        #print(" forbiddenTuple .....", forbiddenTuple)
        if forbiddenTuple != "Exit":
          L.append(forbiddenTuple)


In [33]:
file_path = "results_queries_kakuro93_cutOff_10.txt"
file_path2 = "results_cpus_tuples_kakuro93_cutOff_10.txt"

with open(file_path, 'w') as file, open(file_path2, 'w') as file2:
    # Write the string to the file
    file.write("cpt,AskRec_UniAcq,AskRec_FindTuple,Ask_FindTuple,Ask_UniAcq,answer\n")
    file2.write("#tuples,#constraints,#cpu\n")
    for k in range(10):
      cpt=0
      start_cpu = time.process_time()
      learned_Tuples, learned_c= fineGrainedAcq()
      end_cpu = time.process_time()
      cpu = end_cpu - start_cpu
      print("number of learned tuples: ", len(learned_Tuples))
      print("number of learned constraints: ", len(learned_c))
      file2.write(f"{len(learned_Tuples)},{len(learned_c)},{cpu}\n")
      for t in learned_Tuples:
        print(f"{t}\n")
      for c in learned_c:
        print(f"{c.rel.operator} on {c.scope}\n")



 cpt  0
c'est un exemple negatif, pouvez expliquer pourquoi?
Affirm that the total of X0, X1, X2, X3, X4, when added, equals 34.
 il s'agit d'une contrainte  Sum
Sum Sum
nous allons comprer les scopes.... ['x0', 'x1', 'x2', 'x3', 'x4', '34']  ... [0, 1, 2, 3, 4]
 bert a réussi ?  True
 cpt  2
c'est un exemple negatif, pouvez expliquer pourquoi?
No two elements in X0, X1, X2, X3, X4 should be identical.
 il s'agit d'une contrainte  all_diff
all_diff all_diff
nous allons comprer les scopes.... ['x0', 'x1', 'x2', 'x3', 'x4']  ... [0, 1, 2, 3, 4]
 bert a réussi ?  True
 cpt  4
c'est un exemple negatif, pouvez expliquer pourquoi?
Check that the combined sum of X6, X7, X8, X9, X10, X11 equals 33.
 il s'agit d'une contrainte  Sum
Sum Sum
nous allons comprer les scopes.... ['x6', 'x7', 'x8', 'x9', 'x1011', 'x', '33']  ... [6, 7, 8, 9, 10, 11]
 bert a réussi ?  False
 nous sommes ds find tuple, et c'est un exemple negatif. Pouvez donner une seule raison? 
Confirm that the entirety of X6, X7, X8

In [34]:
"""
0: {"constraint": "all_diff", "template":"templates_all_diff", "n_params":10},
    1: {"constraint": "<", "template":"templates_param1_strictely_lt_param2", "n_params":2},
    2: {"constraint": ">", "template":"templates_param1_strictely_gt_param2", "n_params":2},
    7: {"constraint": "==", "template":"templates_equal_constraint", "n_params":2},
    8: {"constraint": "!=", "template":"templates_different_than", "n_params":2},
    9: {"constraint": "<=", "template":"templates_less_than", "n_params":2},
    10: {"constraint": ">=", "template":"templates_greater_than", "n_params":2},
    12: {"constraint": "Sum", "template":"sum_params_equal_param", "n_params":10},
    13: {"constraint": "O_val_=", "template":"templates_equal_constraint", "n_params":2},
    14: {"constraint": "O_val_!=", "template":"templates_different_than", "n_params":2},
    15: {"constraint": "||=val", "template":"abs_val_p1_minus_p2_equal_p_3", "n_params":3},
    16: {"constraint": "||!=val", "template":"abs_val_param1_minus_param_2_diff_param3", "n_params":3}
"""

# templates acquisition

templates_all_diff = [
    "Each element in {params} should be unique.",
    "Ensure uniqueness among the elements of {params}.",
    "All items in {params} must differ from each other.",
    "Ensure that each value in {params} stands alone.",
    "Distinctness is required for all elements in {params}.",
    "No two elements in {params} should be identical.",
    "Each of {params} should have a unique value.",
    "Avoid any duplication within {params}.",
    "There should be no common elements in {params}.",
    "Ensure that every element in {params} is distinct.",
    "Each entry in {params} must be individual and distinct.",
    "Guarantee that every one of {params} is unique.",
    "Maintain distinct values across {params}.",
    "No element should repeat within {params}.",
    "Every part of {params} must be exclusively its own.",
    "Insist on non-repetitive elements in {params}.",
    "Uphold uniqueness in each aspect of {params}.",
    "Enforce distinctiveness among {params}.",
    "Assure no overlaps in {params}.",
    "Demand individuality in each component of {params}."
]


sum_params_equal_param = [
    "Confirm that the combined total of {params} precisely matches {param}.",
    "Ascertain that the aggregate sum of {params} is exactly equal to {param}.",
    "Make sure that the total amount when {params} are summed up is {param}.",
    "Confirm that the combined total of {params} is {param}.",
    "Make sure the aggregate of {params} equals {param}.",
    "Check that the sum total of {params} amounts to {param}.",
    "Ascertain that {params}, when added, equal {param}.",
    "Ensure the cumulative amount of {params} reaches {param}.",
    "Guarantee that the total sum of {params} is precisely {param}.",
    "The total value of {params} should match {param}.",
    "Validate that the sum of {params} is consistent with {param}.",
    "Affirm that combining {params} totals to {param}.",
    "Be sure that the addition of {params} equals {param}.",
    "Ensure that {params}, summed up, result in {param}.",
    "Make certain the total calculation of {params} is {param}.",
    "The collective amount of {params} must tally to {param}.",
    "Confirm that the sum of all {params} equals {param}.",
    "The overall total of {params} should be exactly {param}.",
    "Verify the cumulative sum of {params} equates to {param}.",
    "Ascertain that the aggregate value of {params} is equal to {param}."
]


templates_param1_strictely_lt_param2 = [
    "{param1} is strictly inferior to {param2}.",
    "{param1} is definitively less than {param2}.",
    "{param1} is unambiguously smaller than {param2}.",
    "{param1} is categorically lower than {param2}.",
    "{param1} is explicitly beneath {param2}.",
    "{param1} is unmistakably below {param2}.",
    "{param1} is clearly under {param2}.",
    "{param1} is decidedly less than {param2}.",
    "{param1} is undeniably beneath {param2}.",
    "{param1} is absolutely lower than {param2}.",
    "{param1} is incontrovertibly smaller than {param2}.",
    "{param1} is inarguably less than {param2}.",
    "{param1} is undebatably beneath {param2}.",
    "{param1} is evidently below {param2}.",
    "{param1} is obviously under {param2}.",
    "{param1} is palpably less than {param2}.",
    "{param1} is manifestly lower than {param2}.",
    "{param1} is patently smaller than {param2}.",
    "{param1} is transparently beneath {param2}.",
    "{param1} is undisputedly below {param2}."
]

templates_param1_strictely_gt_param2 = [
    "{param1} is markedly higher than {param2}.",
    "{param1} towers above {param2}.",
    "{param1} stands out as greater than {param2}.",
    "{param1} holds a distinct advantage over {param2}.",
    "{param1} is considerably more than {param2}.",
    "{param1} surpasses {param2} significantly.",
    "{param1} is notably above {param2}.",
    "{param1} is profoundly greater than {param2}.",
    "{param1} is substantially higher than {param2}.",
    "{param1} exceeds {param2} clearly.",
    "{param1} is visibly more than {param2}.",
    "{param1} is outstandingly above {param2}.",
    "{param1} is demonstrably larger than {param2}.",
    "{param1} clearly outstrips {param2}.",
    "{param1} is significantly beyond {param2}.",
    "{param1} vastly outdoes {param2}.",
    "{param1} is majorly above {param2}.",
    "{param1} is emphatically greater than {param2}.",
    "{param1} is dominantly higher than {param2}.,"
    "{param1} is strictly superior to {param2}."
    ]

templates_different_than=[
    "{param1} varies from {param2}.",
    "{param1} and {param2} are not identical.",
    "{param1} diverges from {param2}.",
    "{param1} stands apart from {param2}.",
    "{param1} is distinct from {param2}.",
    "{param1} contrasts with {param2}.",
    "{param1} is not comparable to {param2}.",
    "{param1} holds no similarity to {param2}.",
    "{param1} is in contrast to {param2}.",
    "{param1} bears no resemblance to {param2}.",
    "{param1} and {param2} are dissimilar.",
    "{param1} does not mirror {param2}.",
    "{param1} is unlike {param2}.",
    "{param1} deviates from {param2}.",
    "{param1} and {param2} are in contrast.",
    "{param1} is not a match with {param2}.",
    "{param1} and {param2} do not align.",
    "{param1} is at variance with {param2}.",
    "{param1} diverges distinctly from {param2}.",
    "{param1} and {param2} are poles apart."
    ]

templates_less_than=[
    "{param1} falls below {param2}.",
    "{param1} is inferior to {param2}.",
    "{param1} ranks under {param2}.",
    "{param1} is beneath {param2}.",
    "{param1} is not as much as {param2}.",
    "{param1} trails {param2}.",
    "{param1} is lower than {param2}.",
    "{param1} doesn't reach the level of {param2}.",
    "{param1} is subordinate to {param2}.",
    "{param1} is behind {param2}.",
    "{param1} is under {param2}.",
    "{param1} should be smaller {param2}.",
    "{param1} is shorter than {param2}.",
    "{param1} is not as high as {param2}.",
    "{param1} is as important as {param2}.",
    "{param1} is secondary to {param2}.",
    "{param1} is outweighed by {param2}.",
    "{param1} is eclipsed by {param2}.",
    "{param1} is less significant than {param2}.",
    "{param1} does not surpass {param2}."
]


templates_greater_than= [
    "{param1} surpasses {param2}.",
    "{param1} is above {param2}.",
    "{param1} outstrips {param2}.",
    "{param1} is more than {param2}.",
    "{param1} exceeds the value of {param2}.",
    "{param1} is superior to {param2}.",
    "{param1} tops {param2}.",
    "{param1} is higher than {param2}.",
    "{param1} overshadows {param2}.",
    "{param1} outdoes {param2}.",
    "{param1} prevails over {param2}.",
    "{param1} is larger than {param2}.",
    "{param1} towers over {param2}.",
    "{param1} outnumbers {param2}.",
    "{param1} leads {param2}.",
    "{param1} dominates over {param2}.",
    "{param1} takes precedence over {param2}.",
    "{param1} outweighs {param2}.",
    "{param1} bests {param2}.",
    "{param1} has the edge over {param2}."
]

sum_params_equal_param= [
    "Confirm that the collective amount of {params} is {param}.",
    "Check that the combined sum of {params} equals {param}.",
    "Ascertain that the aggregate of {params} reaches {param}.",
    "Make sure the total calculation of {params} amounts to {param}.",
    "Ensure that the cumulative value of {params} matches {param}.",
    "Validate that the sum total of {params} is equivalent to {param}.",
    "Guarantee that the overall total of {params} is equal to {param}.",
    "Be certain that adding up {params} yields {param}.",
    "Affirm that the total of {params}, when added, equals {param}.",
    "Establish that the sum of all {params} is exactly {param}.",
    "Ensure the complete sum of {params} tallies up to {param}.",
    "Double-check that the aggregate sum of {params} is {param}.",
    "See to it that the total of {params} equates to {param}.",
    "Confirm that the entirety of {params} sums up to {param}.",
    "Insist that the cumulative effect of {params} totals {param}.",
    "Make certain that the overall effect of {params} is {param}.",
    "Determine that the complete total of {params} matches {param}.",
    "Reconfirm that the sum of {params} aligns with {param}.",
    "Substantiate that the total value of {params} is {param}.",
    "Verify the final sum of {params} equates to {param}."
    ]

abs_val_p1_minus_p2_equal_p_3=[
    "Check the absolute value of the difference between {param1} and {param2} equals {param3}.",
    "Confirm that the absolute difference between {param1} and {param2} is exactly {param3}.",
    "Check that the non-negative difference of {param1} and {param2} totals to {param3}.",
    "Verify that the positive distance between {param1} and {param2} measures {param3}.",
    "Ascertain that the absolute magnitude of the difference from {param1} to {param2} is {param3}.",
    "Make sure the absolute numerical gap between {param1} and {param2} is {param3}.",
    "Determine that the absolute value of subtracting {param2} from {param1} is {param3}.",
    "Validate the absolute distance from {param1} to {param2} as {param3}.",
    "Guarantee that the absolute separation between {param1} and {param2} equals {param3}.",
    "Affirm that the absolute variance of {param1} and {param2} is {param3}.",
    "Establish the absolute numerical difference of {param1} from {param2} as {param3}.",
    "Ensure the absolute deviation between {param1} and {param2} matches {param3}.",
    "Confirm that the absolute level of distinction between {param1} and {param2} is {param3}.",
    "Check the absolute numerical disparity of {param1} and {param2} against {param3}.",
    "Substantiate that the absolute measurement between {param1} and {param2} is {param3}.",
    "Reaffirm that the absolute value representing the difference of {param1} and {param2} is {param3}.",
    "Certify the absolute quantitative difference between {param1} and {param2} as {param3}.",
    "Make certain the absolute numerical divergence between {param1} and {param2} equals {param3}.",
    "Verify that the absolute value of the gap from {param1} to {param2} is {param3}.",
    "Ensure the absolute numerical contrast between {param1} and {param2} adheres to {param3}."
]


abs_val_param1_minus_param_2_diff_param3=[
    "Verify that the absolute value of the gap between {param1} and {param2} is not {param3}.",
    "Check that the absolute difference of {param1} and {param2} does not equal {param3}.",
    "Ascertain that the non-negative difference between {param1} and {param2} is not {param3}.",
    "Make sure the absolute disparity from {param1} to {param2} is unlike {param3}.",
    "Determine that the absolute value of {param1} minus {param2} is not equivalent to {param3}.",
    "Validate that the positive distance of {param1} from {param2} is not {param3}.",
    "Affirm that the absolute numerical difference between {param1} and {param2} isn't {param3}.",
    "Ensure that the absolute magnitude of difference from {param1} to {param2} isn't the same as {param3}.",
    "Guarantee that the absolute value of the discrepancy between {param1} and {param2} isn't {param3}.",
    "Establish that the absolute distance between {param1} and {param2} does not match {param3}.",
    "Confirm the absolute numerical gap between {param1} and {param2} is distinct from {param3}.",
    "Reaffirm that the absolute distance of {param1} and {param2} is not aligned with {param3}.",
    "Ensure the absolute deviation of {param1} from {param2} doesn't equal {param3}.",
    "Certify that the absolute numerical disparity between {param1} and {param2} isn't {param3}.",
    "Make certain the absolute value of the contrast between {param1} and {param2} isn't {param3}.",
    "Check that the absolute measurement of difference between {param1} and {param2} is not {param3}.",
    "Verify that the absolute separation from {param1} to {param2} is other than {param3}.",
    "Substantiate that the absolute separation of {param1} and {param2} doesn't mirror {param3}.",
    "Ensure the absolute numerical distance of {param1} from {param2} is not identical to {param3}.",
    "Confirm that the absolute value representing the differential of {param1} and {param2} diverges from {param3}."
]


In [35]:

# templates sysnthetic data !!
"""
templates_all_diff = [
    "Ensure that {params} are all different.",
    "No matching values allowed among {params}.",
    "Keep {params} separate from each other.",
    "Make sure {params} don't overlap.",
    "{params} should not share any common values.",
    "Avoid repetitions in {params}.",
    "None of {params} should be the same.",
    "{params} must have distinct values.",
    "All values of {params} must be unique.",
    "Ensure that there are no duplicates within {params}.",
    "Prevent any similarities in {params}.",
    "Disallow any identical values in {params}.",
    "Insist on distinct values within {params}.",
    "Prohibit matching values among {params}.",
    "Guarantee that {params} have no shared values.",
    "Make it a requirement that {params} are distinct.",
    "Mandate the uniqueness of values within {params}.",
    "Enforce distinctness in {params}.",
    "Impose a restriction on {params} to be all different.",
    "Enjoin {params} from sharing any values.",
    "Impel {params} to have no duplicates.",
    "Safeguard the uniqueness of values in {params}.",
    "Assert that {params} have no common values.",
    "Insist upon distinct values within {params}.",
    "Secure the non-repetition of values in {params}.",
    "Command that {params} do not coincide.",
    "Make it a rule that {params} have distinct values.",
    "Oblige {params} to be different from each other.",
    "Require that {params} do not overlap.",
    "Stipulate that {params} should not share values.",
    "Demand the absence of repetitions in {params}.",
    "Specify that {params} must be unique.",
    "State that all values of {params} must be distinct.",
    "Assert that {params} should be all distinct.",
    "Emphasize the distinctness of values within {params}.",
    "Highlight the importance of uniqueness in {params}.",
    "Underline the requirement that {params} have no common values.",
    "Express the necessity for distinct values within {params}.",
    "Communicate the need to prevent repetitions in {params}.",
    "Affirm that {params} must not coincide.",
    "Declare that {params} should have unique values.",
    "Acknowledge the importance of non-overlapping values in {params}.",
    "Admonish against the sharing of values within {params}.",
    "Promote the avoidance of repetitions in {params}.",
    "Reiterate the importance of uniqueness within {params}.",
    "Reinforce the requirement for distinct values in {params}.",
    "Stress the necessity for {params} to be all different.",

    "Ensure that each of {params} is distinct.",
    "Make certain that all {params} are uniquely different.",
    "Verify that none of the {params} are identical.",
    "Ascertain that every one of the {params} varies.",
    "Confirm that {params} are all uniquely individual.",
    "Guarantee that each among {params} is disparate.",
    "Ensure no two of {params} are the same.",
    "Check that every element in {params} is dissimilar.",
    "See to it that all of {params} are distinctly separate.",
    "Affirm that each of the {params} stands out as different.",
    "Validate that all items in {params} are non-identical.",
    "Confirm that every single one of {params} is unique.",
    "Ensure that {params} exhibit complete dissimilarity.",
    "Ascertain the uniqueness of each in {params}.",
    "Make sure no element in {params} duplicates another.",
    "Verify the distinctness of each of {params}.",
    "Double-check that all {params} are individually different.",
    "Insist that each of the {params} is singular in identity.",
    "Maintain that every one of {params} is distinctively separate.",
    "Demand that all within {params} are differently characterized.",
    "Ensure absolute uniqueness among {params}.",
    "Check for distinct characteristics in each of {params}.",
    "Affirm the non-similarity of all {params}.",
    "Certify that each of {params} differs from the rest.",
    "Guarantee the individuality of each element in {params}.",
    "Make certain no two of {params} are alike.",
    "Ensure a distinct identity for each of {params}.",
    "Verify the individual distinctness of {params}.",
    "Ascertain that {params} have no duplicates.",
    "Confirm that every item in {params} is diversely unique.",
    "Ensure the exclusivity of each in {params}.",
    "Check that each of {params} has a unique attribute.",
    "Make sure all of {params} are distinctively characterized.",
    "Guarantee that no element of {params} repeats.",
    "See to it that {params} are all individually unique.",
    "Demand the distinctiveness of each in {params}.",
    "Affirm that each of {params} has its own uniqueness.",
    "Validate the non-repetition among {params}.",
    "Certify that all of {params} stand apart.",
    "Ascertain the individual uniqueness of each in {params}.",
    "Ensure no overlap among the elements of {params}.",
    "Check for the individuality of each within {params}.",
    "Confirm the separate identity of each of {params}.",
    "Make certain that {params} are all singularly different.",
    "Verify that each item in {params} has its distinct characteristic.",
    "Double-check the uniqueness of each among {params}.",
    "Insist on distinctiveness for each of {params}.",
    "Maintain the separate nature of all {params}.",
    "Demand a unique aspect for each of {params}.",
    "Affirm the diversity among {params}."
]


sum_params_equal_param = [
    "Ensure the total of {params} equals {param}.",
    "The sum of {params} must be exactly {param}.",
    "Verify that adding {params} together results in {param}.",
    "It is required that {params} combined equal {param}.",
    "Confirm that the aggregate of {params} is {param}.",
    "The collective amount of {params} should match {param}.",
    "Ensure that the cumulative total of {params} amounts to {param}.",
    "The total sum of {params} needs to be precisely {param}.",
    "Verify the combined value of {params} equals {param}.",
    "Ascertain that summing up {params} yields {param}.",
    "The overall total of {params} must equate to {param}.",
    "Confirm that the joint sum of {params} is exactly {param}.",
    "Ensure the accumulated amount of {params} is {param}.",
    "The collective sum of {params} should be {param}.",
    "Verify the total addition of {params} equals {param}.",
    "It's essential that {params} together total {param}.",
    "Confirm the combined sum of {params} reaches {param}.",
    "The entire sum of {params} must tally up to {param}.",
    "Ensure the total of all {params} is precisely {param}.",
    "The grand total of {params} should equal {param}.",
    "Verify the sum total of {params} is {param}.",
    "Ascertain the full total of {params} equals {param}.",
    "The complete sum of {params} must be {param}.",
    "Confirm that the overall addition of {params} is {param}.",
    "Ensure the sum calculation of {params} matches {param}.",
    "The additive total of {params} should equate to {param}.",
    "Verify the entire aggregation of {params} equals {param}.",
    "It's necessary that the full addition of {params} is {param}.",
    "Confirm the collective calculation of {params} totals {param}.",
    "{param} must be the total of {params}.",
    "{param} should equal the sum of {params}.",
    "The value of {param} is the collective sum of {params}.",
    "{param} is required to match the total of {params}.",
    "The exact amount of {param} should be the sum of {params}.",
    "{param} needs to be the combined total of {params}.",
    "The figure {param} must match the aggregate of {params}.",
    "{param} should represent the cumulative sum of {params}.",
    "For {param}, the sum of {params} is essential.",
    "Equating {param} with the total of {params} is necessary.",
    "The amount {param} should be the result of adding {params}.",
    "{param} is the required total of {params}.",
    "The sum of {params} should precisely equal {param}.",
    "{param} must be the end result of summing {params}.",
    "Aim for {param} to be the collective total of {params}.",
    "The combination of {params} must equal {param}.",
    "{param} equals the total sum of {params}.",
    "Summing {params} should yield {param}.",
    "The target of {param} is the sum of {params}.",
    "{param} should be achieved by adding {params}.",
    "The count of {param} equals the combined value of {params}.",
    "Make sure {param} is the total derived from {params}.",
    "{param} represents the sum total of {params}.",
    "The desired {param} is the total of {params}.",
    "Ensure {param} equals the addition of {params}.",
    "The goal is for {param} to be the sum of {params}.",
    "For a total of {param}, sum up {params}.",
    "The aim is to have {param} as the total from {params}.",
    "{param} should be the sum result of {params}.",
    "Achieve {param} by summing the values of {params}.",

    "Guarantee the sum of {params} is in accordance with {param}."	,
"See to it that the total of {params} is in harmony with {param}."	,
"Affirm the aggregate sum of {params} matches {param}."	,
"Validate the combined total of {params} is in balance with {param}."	,
"Confirm the collective value of {params} is equal to {param}."	,
"Make sure the total sum of {params} mirrors {param}."	,
"Ensure the overall amount of {params} aligns precisely with {param}."	,
"Verify that the sum total of {params} is in accordance with {param}."	,
"Ascertain that the aggregate value of {params} is congruent to {param}."	,
"Check if the combined total of {params} equals exactly {param}."	,
"Guarantee that the sum of {params} aligns accurately with {param}."	,
"See that the collective amount of {params} matches the exact value of {param}."	,
"Ensure the joint total of {params} complies with {param}."	,
"Make certain that the combined sum of {params} reaches {param}."	,
"Validate that the sum of {params} is on par with {param}."	,
"Confirm the equality of the total of {params} and {param}."	,
"Verify the total sum of {params} is equivalent to {param}."	,
"Ascertain the combined value of {params} matches exactly {param}."	,
"Check the total of {params} against the standard of {param}."	,
"Double-check that the sum of {params} is congruent with {param}."	,
"Guarantee the overall sum of {params} is in line with {param}."	,
"See to it the collective total of {params} equals {param}."	,
"Affirm the sum of {params} is identical to {param}."	,
"Validate the aggregate of {params} is in harmony with {param}."	,
"Confirm the total of {params} equates to {param}."	,
"Verify the combined amount of {params} corresponds to {param}."	,
"Ascertain the total sum of {params} is congruent with {param}."	,
"Ensure the aggregate value of {params} is in alignment with {param}."	,
"Make sure the sum of {params} matches the value of {param}."	,
"Check the total figure of {params} against {param}."	,
"Guarantee the combined sum of {params} equals {param}."	,
"See that the total of {params} is in accordance with {param}."	,
"Make certain the sum of {params} is in concordance with {param}."	,
"Validate that the overall total of {params} is equal to {param}."	,
"Confirm the cumulative amount of {params} equals {param}."	,
"Verify the joint sum of {params} is the same as {param}."	,
"Ascertain the combined total of {params} is consistent with {param}."	,
"Check the total sum of {params} for equivalence with {param}."	,
"Double-check the sum of {params} for alignment with {param}."	,
"Guarantee the aggregate total of {params} coincides with {param}."	,
"Ensure the overall amount of {params} matches {param}."	,
"See to it that the total of {params} complies with {param}."	,
"Affirm the collective sum of {params} is in sync with {param}."	,
"Validate the combined amount of {params} is equivalent to {param}."	,
"Confirm the sum total of {params} is in agreement with {param}."	,
"Verify the total of {params} is in congruence with {param}."	,
"Ascertain that the aggregate of {params} meets the measure of {param}."	,
"Make sure the total sum of {params} is in line with {param}."	,
"Check that the combined value of {params} is equal to {param}."	,
"Guarantee the collective total of {params} aligns with {param}."	,
"See that the joint amount of {params} matches {param}."	,
"Make certain the total of {params} equals the criteria of {param}."	,
"Validate the sum of {params} for compliance with {param}."	,
"Confirm the overall total of {params} is consistent with {param}."	,
"Verify the cumulative sum of {params} is on target with {param}."	,

]



# OK VERIFIED  !!!!
abs_val_param1_minus_param_2_diff_param3 = [
    "Ensure that {param3} is not equal to the absolute difference between {param1} and {param2}.",
    "Maintain the condition where {param3} is never the same as the absolute value of the difference between {param1} and {param2}.",
    "Consistently verify that {param3} does not match the absolute value of the difference between {param1} and {param2}.",
    "It's an unbreakable rule that {param3} is never the same as the absolute value of the result when you subtract {param2} from {param1}.",
    "Ensure that {param3} is not the same as the absolute value of the result when you subtract {param2} from {param1}.",
    "In this particular context, the non-negotiable requirement is that {param3} is not equal to the absolute difference between {param2} and {param1}.",
    "Consistently maintain the condition where {param3} is not identical to the absolute value of the result of subtracting {param2} from {param1}.",
    "Reject any value where {param3} is identical to the absolute value of the result of subtracting {param2} from {param1}.",
    "It's an unbreakable rule that {param3} is never the same as the absolute value of the result when you subtract {param1} from {param2}.",
    "Ensure that {param3} is not the same as the absolute value of the result of subtracting {param1} from {param2}.",
    "In this particular context, the non-negotiable requirement is that {param3} is not equal to the absolute difference between {param1} and {param2}.",
    "Consistently verify that {param3} does not match the absolute value of subtracting {param1} from {param2}.",
    "It's an unbreakable rule that {param3} is never the same as the absolute value of the result when you subtract {param2} from {param1}.",
    "Verify that {param3} is not equal to the absolute value of {param1} minus {param2}.",
    "In this particular context, the non-negotiable requirement is that {param3} is not equal to the absolute difference between {param1} and {param2}.",
    "Verify that {param3} is not equal to the absolute value of {param1} minus {param2}.",
    "Verify that the absolute value of {param1} minus {param2} does not equal {param3}.",
    "Ensure that the absolute difference between {param1} and {param2} is not the same as {param3}.",
    "In this context, it's crucial that the absolute value {param1} minus {param2} is not identical to {param3}.",
    "Consistently ensure that the absolute value of {param1} minus {param2} is never equivalent to {param3}.",
    "Reject any scenario where the absolute value of {param1} minus {param2} equals {param3}.",
    "Do not accept the case where the absolute value of result of subtracting {param1} from {param2} is identical to {param3}.",
    "Always ensure that the inequality of the absolute value of the result of subtracting {param1} from {param2} and {param3} is upheld: the two should never be the same.",
    "Verify that the absolute value of the result of subtracting {param1} from {param2} is not equal to {param3}.",
    "Ensure that {param1} and {param2} are not separated by exactly {param3}.",
    "The gap between {param1} and {param2} must not be {param3}.",
    "It's crucial that {param1} and {param2} do not differ by exactly {param3}.",
    "The value {param3} should not represent the span from {param1} to {param2}.",
    "{param1} and {param2} should have a difference other than {param3}.",
    "Make sure the numerical distance between {param1} and {param2} is not {param3}.",
    "The range separating {param1} and {param2} cannot be {param3}.",
    "The measure between {param1} and {param2} should not exactly be {param3}.",
    "Avoid a situation where {param1} is {param3} away from {param2}.",
    "The discrepancy between {param1} and {param2} should not equal {param3}.",
    "Ensure the numerical gap between {param1} and {param2} isn't {param3}.",
    "{param1} and {param2} must not have a {param3} unit disparity.",
    "The separation in value between {param1} and {param2} should not be {param3}.",
    "Keep {param1} and {param2} from having a {param3} value difference.",
    "The distance measured from {param1} to {param2} must not total {param3}.",
    "The gap between {param1} and {param2} must not be {param3}.",
    "The measure between {param1} and {param2} should not exactly be {param3}.",
    "Avoid a situation where {param1} is {param3} away from {param2}.",
    "The discrepancy between {param1} and {param2} should not equal {param3}.",
    "Ensure the numerical gap between {param1} and {param2} isn't {param3}.",
    "{param1} and {param2} must not have a {param3} unit disparity.",
    "Ensure that the absolute difference between {param1} and {param2} is not {param3}.",
    "Check that {param1} minus {param2}, in absolute terms, does not equate to {param3}.",
    "Confirm that taking the absolute value of {param1} - {param2} does not result in {param3}.",
    "Ascertain that the absolute subtraction of {param1} from {param2} isn't {param3}.",
    "Make sure the absolute numerical difference of {param1} and {param2} is not {param3}.",
    "Affirm that the absolute value of {param1} less {param2} does not add up to {param3}.",
    "Verify the non-equality of the absolute difference of {param1} and {param2} to {param3}.",
    "Check against the absolute value of {param1} - {param2} being {param3}.",
    "Ascertain that the absolute disparity between {param1} and {param2} doesn't equal {param3}.",
    "Ascertain that {param1} subtracted from {param2}, absolutely, is not {param3}.",
    "Confirm that the net absolute difference of {param1} and {param2} isn't {param3}.",
    "Validate that the absolute disparity between {param1} and {param2} doesn't equal {param3}.",
    "Guarantee that the absolute value of the difference between {param1} and {param2} isn't {param3}.",
    "See to it that the absolute calculation of {param1} minus {param2} does not result in {param3}.",
    "Make sure the absolute numerical difference of {param1} and {param2} is not {param3}.",
    "Affirm that the absolute value of {param1} less {param2} does not add up to {param3}.",
    "Verify the non-equality of the absolute difference of {param1} and {param2} to {param3}.",
    "Check against the absolute value of {param1} - {param2} being {param3}.",
    "Verify that the result of the absolute subtraction of {param1} from {param2} is not {param3}.",
    "Ensure that the absolute value of {param1} minus {param2} is not {param3}.",
    "Ensure that the absolute value differential of {param1} and {param2} does not equate to {param3}.",
    "Confirm that the absolute distance between {param1} and {param2} is not the same as {param3}.",
    "Ascertain that the absolute variance of {param1} minus {param2} is not {param3}.",
    "Make certain that the absolute deviation of {param1} from {param2} is not equal to {param3}.",
    "Verify that the modulus of {param1} minus {param2} does not match {param3}.",
    "Check that the absolute magnitude of {param1} - {param2} isn't {param3}.",
    "Ensure that the absolute value result of {param1} less {param2} is not {param3}.",
    "Affirm that the absolute numerical distance between {param1} and {param2} isn't {param3}.",
    "Validate that the absolute subtraction result of {param1} and {param2} is not {param3}.",
    "Guarantee that the absolute figure of {param1} minus {param2} does not amount to {param3}.",
    "Ascertain that the outcome of |{param1} - {param2}| is not {param3}.",
    "Confirm the non-equality of the absolute difference between {param1} and {param2} to {param3}.",
    "Check for the absence of equality in the absolute value of {param1} minus {param2} to {param3}.",
    "Make sure that the absolute numerical gap of {param1} and {param2} is not {param3}.",
    "Verify that the absolute level of difference between {param1} and {param2} isn't {param3}.",
    "Ensure the non-coincidence of the absolute value of {param1} - {param2} with {param3}.",
    "Ascertain that the absolute numerical disparity between {param1} and {param2} does not equal {param3}."

]


# VERIFIED OK !!!!!!!!



abs_val_p1_minus_p2_equal_p_3 = [
    "Ensure that the absolute difference between {param1} and {param2} equals {param3}.",
    "The condition is that the magnitude of the difference between {param1} and {param2} should be {param3}.",
    "Verify that the absolute value of ({param1} subtracted from {param2}) is {param3}.",
    "Make sure that the absolute difference between {param1} and {param2} matches {param3}.",
    "Maintain the absolute difference between {param1} and {param2} as {param3} at all times.",
    "It's non-negotiable that the absolute value of the result of subtracting {param1} from {param2} matches {param3}.",
    "Ensure that the absolute difference between {param1} and {param2} remains {param3}.",
    "The cardinal rule is that the absolute difference between {param1} and {param2} must be {param3}.",
    "Do not allow the result of subtracting {param1} from {param2} in absolute value to be unequal to {param3} under any circumstances.",
    "The unbreakable rule is that the absolute difference between {param1} and {param2} cannot deviate from {param3}."
    "Ensure that the absolute difference between {param1} and {param2} is equivalent to {param3}.",
    "Verify that the absolute value of the subtraction of {param2} from {param1} equals {param3}.",
    "Always ensure that the absolute value of the result of {param2} subtracted from {param1} is not different from {param3}.",
    "The cardinal rule is that the absolute difference between {param2} and {param1} must be {param3}.",
    "Do not allow the result of {param2} subtracted from {param1} in absolute value to be unequal to {param3} under any circumstances.",
    "The unbreakable rule is that the absolute difference between {param2} and {param1} cannot deviate from {param3}.",
    "Make certain that the absolute value of ({param1} with {param2} subtracted) is equal to {param3}.",
    "Ensure that the absolute value of the difference between {param1} and {param2} is the same as {param3}.",
    "Make sure that the absolute difference between {param1} and {param2} matches {param3}.",
    "Always ensure that the absolute value of the result of {param1} subtracted from {param2} is not different from {param3}.",
    "Do not allow the result of {param1} subtracted from {param2} in absolute value to be unequal to {param3} under any circumstances.",
    "The unbreakable rule is that the absolute difference between {param1} and {param2} cannot deviate from {param3}.",
    "Confirm that the absolute value of the difference between {param1} and {param2} precisely equals {param3}.",
    "Double-check that the absolute value of the subtraction of {param2} from {param1} is {param3}.",
    "Ensure that the absolute difference between {param2} and {param1} consistently matches {param3}.",
    "The unalterable rule is that the absolute difference between {param2} and {param1} cannot deviate from {param3}.",
    "Always ensure that the absolute value of the result of {param2} subtracted from {param1} is not different from {param3}.",
    "Maintain the absolute difference between {param2} and {param1} as {param3} without exception.",
    "It's non-negotiable that the absolute value of the result of {param2} subtracted from {param1} matches {param3}.",
    "Ensure that the absolute difference between {param2} and {param1} always coincides with {param3}."
    "Ensure that {param3} is exactly the distance between {param1} and {param2}.",
    "The value of {param3} must match the distance between {param1} and {param2}.",
    "It is required that {param3} equals the distance separating {param1} and {param2}.",
    "Make sure {param3} reflects the exact distance between {param1} and {param2}.",
    "{param3} should equal the distance between {param1} and {param2}.",
    "The amount {param3} must be the distance from {param1} to {param2}.",
    "{param3} needs to represent the distance separating {param1} and {param2}.",
    "The figure {param3} should match the distance between {param1} and {param2}.",
    "Ensure the distance between {param1} and {param2} is exactly {param3}.",
    "The distance measure between {param1} and {param2} should be {param3}.",
    "Ensure the gap measured between {param1} and {param2} is {param3}.",
    "{param3} has to be the exact span between {param1} and {param2}.",
    "The length between {param1} and {param2} must equal {param3}.",
    "Ensure {param3} represents the distance between {param1} and {param2}.",
    "The separation between {param1} and {param2} should be {param3}.",
    "The distance covered from {param1} to {param2} must be {param3}.",
    "{param3} should be the length spanning between {param1} and {param2}.",
    "Make sure the separation distance between {param1} and {param2} equals {param3}.",
    "The interval between {param1} and {param2} must equate to {param3}.",
    "The reach between {param1} and {param2} needs to be exactly {param3}.",
    "Set the distance from {param1} to {param2} to exactly {param3}.",
    "Ensure the span between {param1} and {param2} matches {param3}.",
    "The distance metric between {param1} and {param2} should be set to {param3}.",
    "Confirm that the interval between {param1} and {param2} is {param3}.",
    "Match the distance between {param1} and {param2} to {param3}.",
    "The range between {param1} and {param2} should precisely be {param3}.",
    "Ensure the length spanned between {param1} and {param2} is {param3}.",
    "{param3} must be the reach from {param1} to {param2}.",
    "The distance value between {param1} and {param2} must be set as {param3}.",
    "Ensure that the absolute difference between {param1} and {param2} equals {param3}.",
    "Make sure the absolute value of the difference between {param1} and {param2} is {param3}.",
    "Confirm that {param3} is the absolute difference of {param1} and {param2}.",
    "Verify that when subtracting {param2} from {param1}, the absolute result is {param3}.",
    "Check that the absolute value of {param1} minus {param2} equals {param3}.",
    "Ascertain that the absolute difference between {param1} and {param2} matches {param3}.",
    "Ensure that {param3} represents the absolute difference between {param1} and {param2}.",
    "Make certain the absolute difference between {param1} and {param2} is precisely {param3}.",
    "Ensure that the absolute difference between {param1} and {param2} equals {param3}.",
    "Make sure the absolute value of the difference between {param1} and {param2} is {param3}.",
    "Confirm that {param3} is the absolute difference of {param1} and {param2}.",
    "Verify that when subtracting {param2} from {param1}, the absolute result is {param3}.",
    "Check that the absolute value of {param1} minus {param2} equals {param3}.",
    "Ascertain that the absolute difference between {param1} and {param2} matches {param3}.",
    "Ensure that {param3} represents the absolute difference between {param1} and {param2}.",
    "Make certain the absolute difference between {param1} and {param2} is precisely {param3}.",
    "Double-check that the absolute gap between {param1} and {param2} is exactly {param3}.",
    "Verify the equality of {param3} with the absolute difference between {param1} and {param2}.",
    "Confirm the equivalence of {param3} and the absolute difference of {param1} and {param2}.",
    "See to it that the absolute discrepancy between {param1} and {param2} amounts to {param3}.",
    "Establish that {param3} is equivalent to the absolute difference between {param1} and {param2}.",
    "Ensure the absolute variation between {param1} and {param2} is equal to {param3}.",
    "Check if the absolute differential between {param1} and {param2} is {param3}.",
    "Affirm that the absolute disparity between {param1} and {param2} corresponds to {param3}.",
    "Validate that the difference between {param1} and {param2}, in absolute terms, equals {param3}.",
    "Confirm the absolute difference of {param1} from {param2} equals {param3}.",
    "Ensure that subtracting {param2} from {param1}, and taking the absolute, yields {param3}.",
    "Guarantee that the absolute divergence between {param1} and {param2} is {param3}.",
    "Ascertain that {param3} is the result of the absolute difference between {param1} and {param2}.",
    "Ensure the absolute delta between {param1} and {param2} equates to {param3}.",
    "Check that {param3} is exactly the absolute difference between {param1} and {param2}.",
    "Verify that the absolute deviation of {param1} from {param2} is precisely {param3}.",
    "Make sure the absolute subtraction of {param2} from {param1} equals {param3}.",
    "Certify that the absolute numerical difference between {param1} and {param2} is {param3}.",
    "Ascertain that {param3} exactly equals the absolute difference between {param1} and {param2}.",
    "Confirm that the absolute value of {param1} less {param2} is {param3}.",
    "Ensure the absolute disjunction between {param1} and {param2} is {param3}.",
    "Verify that the net absolute difference between {param1} and {param2} equals {param3}.",
    "Check that the absolute contrast between {param1} and {param2} equates to {param3}.",
    "Determine if the absolute differential between {param1} and {param2} is {param3}.",
    "Make certain that the exact absolute difference between {param1} and {param2} is {param3}.",
    "Assure that the absolute variance between {param1} and {param2} is equal to {param3}.",
    "Validate that the precise absolute difference between {param1} and {param2} is {param3}.",
    "Establish the exact equivalence of {param3} to the absolute difference of {param1} and {param2}.",
    "Ensure the absolute numerical gap between {param1} and {param2} is {param3}.",
    "Confirm that the total absolute difference between {param1} and {param2} equals {param3}.",
    "Verify that the absolute disparity in value between {param1} and {param2} is {param3}."
    ]



templates_equal_constraint = [
    "{param1} is equal to {param2}",
    "{param1} and {param2} are the same",
    "It is required that {param1} equals {param2}",
    "{param1} should match {param2}",
    "{param2} must be identical to {param1}",
    "In this scenario, {param1} and {param2} are equal",
    "Please confirm that {param1} equals {param2}",
    "Make sure {param1} is identical to {param2}",
    "If you see {param1}, it should mean {param2} is the same",
    "No exceptions allowed: {param1} equals {param2}",
    "{param1} and {param2} have to be the same",
    "{param1} and {param2} must be identical",
    "To satisfy the condition, {param1} should match {param2}",
    "{param2} should always be the same as {param1}",
    "{param1} and {param2} should match",
    "{param2} is required to be identical to {param1}",
    "{param1} should consistently match {param2}",
    "Verify that {param1} equals {param2}",
    "Always ensure that {param2} and {param1} are the same",
    "{param1} is intended to be equal to {param2}",
    "{param2} must always match {param1}",
    "Always check that {param1} and {param2} are the same",
    "Ensure that {param1} and {param2} are always equal",
    "{param1} and {param2} emerge as equal values",
    "{param1} should always maintain equality with {param2}",
    "Confirm that {param1} and {param2} match",
    "The expectation is that {param2} is equal to {param1}",
    "Maintain {param2} as an equal value to {param1}",
    "It's mandatory for {param1} and {param2} to match",
    "{param1} and {param2} should always be equal",
    "Never compromise on {param1} matching {param2}",
    "In this scenario, {param1} and {param2} must remain equal",
    "The golden principle is that {param1} equals {param2}",
    "Never deviate from {param2} matching {param1}",
    "Make sure that {param1} consistently matches {param2}",
    "Under no circumstances should {param2} not be the same as {param1}",
    "Check that {param2} and {param1} maintain equality",
    "Under all circumstances, {param2} should be the same as {param1}",
    "The primary condition is that {param1} and {param2} match",
    "No exceptions should be made regarding {param1} and {param2} being equal",
    "It is imperative that {param1} and {param2} match",
    "The key requirement is that {param1} and {param2} are the same",
    "The golden standard is that {param1} and {param2} match",
    "Never allow {param2} to not match {param1}",
    "Double-check that {param1} and {param2} match",
    "Ensure that {param2} equals {param1}",
    "Always maintain that {param2} and {param1} are equal values",
    "Keep {param2} equal to {param1} at all times",
    "In this scenario, {param2} and {param1} should match",
    "Ensure that {param1} matches {param2}",
    "The requirement is that {param1} and {param2} match",
    "Maintain {param1} and {param2} as equal values",
    "Confirm that {param1} and {param2} are the same",
    "The golden rule is that {param1} and {param2} match",
    "Under no circumstances should {param1} not equal {param2}",
    "Double-check that {param2} is the same as {param1}",
    "Ensure {param2} matches {param1}",
    "Confirm that {param1} and {param2} match",
    "The expectation is that {param2} is equal to {param1}",
    "Maintain {param2} as an equal value to {param1}",
    "It's mandatory for {param1} and {param2} to match",
    "{param1} and {param2} should always be equal",
    "Never compromise on {param1} matching {param2}",
    "In this scenario, {param1} and {param2} must remain equal",
     "{param1} is equal to {param2}.",
    "{param1} equals {param2}.",
    "{param1} is the same as {param2}.",
    "The value of {param1} is identical to {param2}.",
    "{param1} and {param2} are equivalent.",
    "There is equality between {param1} and {param2}.",
    "{param1} matches {param2}.",
    "{param1} is equivalent to {param2}.",
    "The magnitude of {param1} equals that of {param2}.",
    "{param1} and {param2} are equal in value.",
    "Equality is observed between {param1} and {param2}.",
    "{param1} is commensurate with {param2}.",
    "There's no difference between {param1} and {param2}.",
    "{param1} holds the same value as {param2}.",
    "A state of equivalence exists between {param1} and {param2}.",
    "{param1} mirrors {param2} in value.",
    "The measure of {param1} is equal to {param2}.",
    "{param1} is on par with {param2}.",
    "No variance exists between {param1} and {param2}.",
    "The numerical value of {param1} is the same as {param2}.",
    "{param1} coincides with {param2}.",
    "{param1} and {param2} hold identical values.",
    "Parity is found between {param1} and {param2}.",
    "{param1} is tantamount to {param2}.",
    "Equivalency is established between {param1} and {param2}.",
    "{param1} corresponds exactly to {param2}.",
    "There's an equal measure between {param1} and {param2}.",
    "{param1} bears the same magnitude as {param2}.",
    "The scale of {param1} is equivalent to that of {param2}.",
    "{param1} is balanced with {param2}.",
    "The quantity of {param1} is akin to {param2}.",
    "In terms of value, {param1} is similar to {param2}.",
    "{param1} aligns with {param2}.",
    "An equal footing is shared by {param1} and {param2}.",
    "{param1} is congruent with {param2}.",
    "The level of {param1} matches {param2}.",
    "Uniformity exists between {param1} and {param2}.",
    "{param1} is akin to {param2}.",
    "There is no disparity between {param1} and {param2}.",
    "Equivalence is the case for {param1} and {param2}.",
    "{param1} is as much as {param2}.",
    "{param1} and {param2} share the same numerical value.",
    "The value of {param1} corresponds to {param2}.",
    "{param1} is comparable to {param2}.",
    "No difference in value between {param1} and {param2}.",
    "{param1} has an equal value to {param2}.",
    "The worth of {param1} is the same as {param2}.",
    "Equally valued: {param1} and {param2}.",
    "The count of {param1} mirrors that of {param2}.",
    "In comparison, {param1} and {param2} are equal."
]

templates_different_than = [
    "{param1} is different from {param2}",
    "{param2} is not the same as {param1}",
    "{param1} and {param2} are distinct",
    "It is required that {param1} and {param2} are not equal",
    "{param1} should not match {param2}",
    "{param2} must not be identical to {param1}",
    "In this scenario, {param1} and {param2} are different",
    "Please confirm that {param1} does not equal {param2}",
    "Check if {param1} is not the same as {param2}",
    "You should have {param1} distinct from {param2}",
    "{param1} should always differ from {param2}",
    "The rule is that {param1} cannot be equal to {param2}",
    "When comparing {param1} and {param2}, they should not match",
    "Make sure {param1} is not identical to {param2}",
    "If you see {param1}, it should mean {param2} is not the same",
    "No exceptions allowed: {param1} different than {param2}",
    "{param1} and {param2} have to be distinct",
    "{param1} and {param2} must not be identical",
    "To satisfy the condition, {param1} should not equal {param2}",
    "In this scenario, {param1} and {param2} should be different",
    "Ensure that {param2} does not equal {param1}",
    "The requirement is that {param1} and {param2} should differ",
    "{param1} should stay distinct from {param2}",
    "When it comes to {param1} and {param2}, they should not match",
    "Check that {param1} and {param2} do not equal",
    "The primary condition is that {param2} cannot be equal to {param1}",
    "Do not accept any value where {param1} and {param2} match",
    "Maintain {param1} and {param2} as distinct values",
    "Confirm that {param1} and {param2} are not the same",
    "The golden rule is that {param1} and {param2} are different",
    "Under no circumstances should {param1} equal {param2}",
    "Double-check that {param2} is not the same as {param1}",
    "Ensure {param2} is different from {param1}",
    "In this case, {param2} should not equal {param1}",
    "{param1} should consistently differ from {param2}",
    "Verify that {param1} does not equal {param2}",
    "Always ensure that {param2} and {param1} are not the same",
    "{param1} is intended to be different from {param2}",
    "{param2} must never match {param1}",
    "The top priority is that {param2} differs from {param1}",
    "It's non-negotiable that {param1} does not equal {param2}",
    "For {param1} and {param2}, the rule is that they are different",
    "{param2} should always remain different from {param1}",
    "The key requirement is that {param1} and {param2} are not the same",
    "The golden standard is that {param1} and {param2} are distinct",
    "Never allow {param2} to match {param1}",
    "Double-check that {param1} and {param2} are distinct",
    "Ensure that {param2} does not equal {param1}",
    "Always maintain that {param2} and {param1} are different values",
    "Keep {param2} distinct from {param1} at all times",
    "In this scenario, {param2} and {param1} should be different",
    "Ensure that {param1} does not match {param2}",
    "The requirement is that {param2} and {param1} should differ",
    "{param2} should never be the same as {param1}",
    "{param1} and {param2} should not match",
    "{param2} is not allowed to be equal to {param1}",
    "Check that {param1} and {param2} do not equal",
    "The primary condition is that {param2} cannot be equal to {param1}",
    "Do not accept any value where {param1} and {param2} match",
    "Maintain {param1} and {param2} as distinct values",
    "Confirm that {param1} and {param2} are not the same",
	  "{param2} should always remain different from {param1}",
    "{param1} is different from {param2}.",
    "{param1} is not the same as {param2}.",
    "{param1} and {param2} are not identical.",
    "There is a difference between {param1} and {param2}.",
    "{param1} differs from {param2}.",
    "{param1} is distinct from {param2}.",
    "Disparity exists between {param1} and {param2}.",
    "{param1} is unlike {param2}.",
    "Inequality exists between {param1} and {param2}.",
    "There's a variance between {param1} and {param2}.",
    "{param1} and {param2} are dissimilar.",
    "{param1} is not equivalent to {param2}.",
    "The values of {param1} and {param2} are not equal.",
    "{param1} does not match {param2}.",
    "There is a distinction between {param1} and {param2}.",
    "{param1} is not akin to {param2}.",
    "A divergence is present between {param1} and {param2}.",
    "{param1} and {param2} do not correspond.",
    "{param1} is not comparable to {param2}.",
    "There's no parity between {param1} and {param2}.",
    "The characteristics of {param1} and {param2} are different.",
    "{param1} and {param2} do not align.",
    "There is a discrepancy between {param1} and {param2}.",
    "{param1} does not equate with {param2}.",
    "The essence of {param1} is different from {param2}.",
    "Dissimilarity is noted between {param1} and {param2}.",
    "{param1} stands in contrast to {param2}.",
    "There's an imbalance between {param1} and {param2}.",
    "{param1} is in disparity with {param2}.",
    "No similarity between {param1} and {param2}.",
    "{param1} diverges from {param2}.",
    "The nature of {param1} is not similar to {param2}.",
    "There's a distinct difference between {param1} and {param2}.",
    "{param1} and {param2} are not congruent.",
    "A clear distinction exists between {param1} and {param2}.",
    "{param1} bears no resemblance to {param2}.",
    "The value of {param1} is unlike that of {param2}.",
    "{param1} and {param2} are not in harmony.",
    "Discrepancies are evident between {param1} and {param2}.",
    "The identity of {param1} is different from {param2}.",
    "There's a clear divergence between {param1} and {param2}.",
    "The properties of {param1} differ from those of {param2}.",
    "Non-identical: {param1} and {param2}.",
    "A lack of similarity is seen between {param1} and {param2}.",
    "The configuration of {param1} is not the same as {param2}.",
    "{param1} and {param2} show unlikeness.",
    "{param1} is not a mirror of {param2}.",
    "Variation is the key between {param1} and {param2}.",
    "{param1} holds a different value from {param2}.",
    "The composition of {param1} contrasts with {param2}."
]


### ok VERIFIED !!!!



templates_param1_strictely_lt_param2 = [
    "{param1} falls strictely short of {param2}.",
    "{param1} is strictely smaller than {param2}.",
    "{param1} trails clearly behind {param2}.",
    "In terms of value, {param1} is clearly inferior to {param2}.",
    "Without question, {param1} takes the backseat to {param2}.",
    "{param1} is decisively surpassed by {param2}.",
    "{param1} is indisputably beneath {param2}.",
    "There's no doubt about it, {param1} ranks lower than {param2}.",
    "{param1} is emphatically smaller than {param2}.",
    "{param1} is exceedingly lower than {param2}.",
    "When it comes to value, {param1} is exceptionally inferior to {param2}.",
    "{param1} is extremely below {param2}.",
    "{param1} is vastly smaller than {param2}.",
    "It's crystal clear that {param1} strictly falls behind {param2}.",
    "In terms of value, {param1} is decisively inferior to {param2}.",
    "{param1} is markedly lower than {param2} in value.",
    "{param1} falls far short of {param2} consistently.",
    "{param1} consistently trails behind {param2}.",
    "{param1} is consistently outperformed by {param2}.",
    "Without question, {param1} is consistently inferior to {param2}.",
    "{param1} is undeniably and consistently ranked lower than {param2}.",
    "In this comparison, {param1} consistently lags behind {param2}.",
    "Reject any scenario where {param1} equals or surpasses {param2}.",
    "{param1} is not a match for {param2}, strictly speaking.",
    "It's an unwavering truth that {param1} is persistently and strictly less than {param2}.",
    "In all circumstances, {param1} is undeniably and consistently inferior to {param2}.",
    "There's no deviation from the fact that {param1} consistently ranks beneath {param2}.",
    "In every instance, it's consistently proven that {param1} resides in a lower position compared to {param2}.",
    "Without question, {param1} consistently takes a subordinate role compared to {param2}.",
    "{param1} consistently remains in the shadow of {param2}.",
    "In terms of value, {param1} consistently falls far behind {param2}.",
    "{param1} is consistently and decisively inferior to {param2}.",
    "Without any variation, {param1} consistently ranks lower than {param2}.",
    "In each and every context, {param1} consistently resides beneath {param2}.",
    "There's no departure from the fact that {param1} consistently falls short of {param2}.",
    "In every scenario, {param1} is consistently and emphatically smaller than {param2}.",
    "It's consistently and undeniably clear that {param1} is strictly less than {param2}.",
    "In all cases, {param1} is undeniably and consistently inferior to {param2}.",
    "Without fail, {param1} consistently ranks beneath {param2}, no exceptions.",
    "In each and every situation, {param1} consistently remains beneath {param2}.",
    "Without question, {param1} consistently and firmly stays in a position of inferiority compared to {param2}.",
    "It's consistently evident that {param1} takes a subordinate role compared to {param2}.",
    "In the context of value, {param1} consistently lags behind {param2}.",
    "Without any deviations, {param1} consistently falls short of {param2}.",
    "It's an undeniable fact that {param1} is persistently and strictly less than {param2}.",
    "In all instances, {param1} consistently and undeniably ranks lower than {param2}.",
    "{param1} is strictly smaller than {param2}.",
    "{param1} is less than {param2}, without exception.",
    "{param1} is invariably smaller than {param2}.",
    "Without exception, {param1} is less than {param2}.",
    "{param1} is always less than {param2}.",
    "In every case, {param1} is smaller than {param2}.",
    "{param1} is consistently less than {param2}.",
    "There's no instance where {param1} is not smaller than {param2}.",
    "{param1} is unfailingly smaller than {param2}.",
    "{param1} is without fail smaller than {param2}.",
    "It is a certainty that {param1} is smaller than {param2}.",
    "{param1} is decidedly smaller than {param2}.",
    "In all situations, {param1} is less than {param2}.",
    "{param1} is definitively less than {param2}.",
    "{param1} is unmistakably smaller than {param2}.",
    "Without doubt, {param1} is smaller than {param2}.",
    "{param1} is undeniably less than {param2}.",
    "It is an absolute that {param1} is smaller than {param2}.",
    "Invariably, {param1} is smaller than {param2}.",
    "{param1} is irrefutably less than {param2}.",
    "There's an absolute difference with {param1} being smaller than {param2}.",
    "{param1} is without a doubt smaller than {param2}.",
    "{param1} is unarguably less than {param2}.",
    "Unconditionally, {param1} is smaller than {param2}.",
    "It's a given that {param1} is smaller than {param2}.",
    "Undoubtedly, {param1} is less than {param2}.",
    "{param1} is indisputably smaller than {param2}.",
    "It is incontrovertible that {param1} is less than {param2}.",
    "{param1} is clearly smaller than {param2}.",
    "It's irrefutable that {param1} is smaller than {param2}.",
    "Evidently, {param1} is less than {param2}.",
    "{param1} is inarguably smaller than {param2}.",
    "Manifestly, {param1} is less than {param2}.",
    "Patently, {param1} is smaller than {param2}.",
    "There's no question that {param1} is smaller than {param2}.",
    "{param1} is plainly less than {param2}.",
    "It's transparent that {param1} is smaller than {param2}.",
    "Unequivocally, {param1} is less than {param2}.",
    "Without controversy, {param1} is smaller than {param2}.",
    "{param1} is palpably less than {param2}.",
    "It's evident that {param1} is smaller than {param2}.",
    "Unquestionably, {param1} is less than {param2}.",
    "It's clear that {param1} is smaller than {param2}.",
    "Beyond dispute, {param1} is less than {param2}.",
    "There's no denying {param1} is smaller than {param2}.",
    "{param1} is unchallengedly less than {param2}.",
    "It's beyond question that {param1} is smaller than {param2}.",
    "Assuredly, {param1} is less than {param2}.",
    "It goes without saying that {param1} is smaller than {param2}.",
    "It is undisputed that {param1} is less than {param2}."
]




templates_less_than = [
    "{param1} is less than {param2}",
    "{param1} is smaller than {param2}",
    "{param1} is inferior to {param2}",
    "It is required that {param1} is less than {param2}",
    "The value of {param1} should be lower than {param2}",
    "Please confirm that {param1} is less than {param2}",
    "For this condition, {param2} must be larger than {param1}",
    "The value of {param1} should be lower than {param2}",
    "Please confirm that {param1} is less than {param2}",
    "Check if {param1} is less than {param2} as required",
    "You should have {param1} being smaller than {param2}",
    "{param1} should always be lesser than {param2}",
     "Always maintain {param1} as the smaller value under {param2}",
    "Keep {param1} lower than {param2} at all times",
    "In this scenario, {param1} should be lesser than {param2}",
    "The requirement is that {param1} should be inferior to {param2}",
    "{param1} should stay behind {param2}",
    "When it comes to {param1} and {param2}, {param2} wins",
    "Check that {param1} is always smaller than {param2}",
    "Do not accept any value where {param1} is not less than {param2}",
    "Maintain {param1} at a lower level than {param2}",
    "Confirm that {param1} takes a back seat to {param2}",
    "It is required that {param1} is less than {param2}",
    "{param1} should consistently fall short of {param2}",
    "Verify that {param1} never surpasses {param2}",
    "Always ensure that {param2} does not fall short of {param1}",
    "{param1} is intended to be less than {param2}",
    "The primary condition is that {param1} should be less than {param2}",
    "Verify that {param1} never surpasses {param2}",
    "Always ensure that {param1} does not outperform {param2}",
    "Under no circumstances should {param1} take the lead over {param2}",
    "It's non-negotiable that {param1} is less than {param2}",
    "For {param1} and {param2}, the rule is that {param2} wins",
    "In any comparison between {param1} and {param2}, {param2} should win",
    "Double-check that {param1} never surpasses {param2}",
    "{param1} is less than {param2}.",
    "{param1} is smaller than {param2}.",
    "{param1} is lower than {param2}.",
    "The value of {param1} is beneath that of {param2}.",
    "{param1} falls short of {param2}.",
    "{param1} is not as much as {param2}.",
    "{param1} is inferior to {param2}.",
    "{param1} ranks below {param2}.",
    "The magnitude of {param1} is less than {param2}.",
    "In comparison, {param1} is less than {param2}.",
    "{param1} is beneath {param2} in value.",
    "{param1} doesn't reach the value of {param2}.",
    "{param1} is lesser compared to {param2}.",
    "{param1} comes short of {param2}.",
    "The number {param1} is smaller than {param2}.",
    "{param1} is not greater than {param2}.",
    "{param1} holds a lower value than {param2}.",
    "{param1} is eclipsed by {param2}.",
    "{param1} falls below {param2}.",
    "The quantity of {param1} is less than {param2}.",
    "{param1} is under {param2}.",
    "{param1} trails behind {param2}.",
    "{param1} is second to {param2}.",
    "When compared, {param1} is less than {param2}.",
    "{param1} is overshadowed by {param2}.",
    "{param1} is not as great as {param2}.",
    "{param1} is at a lower level than {param2}.",
    "The scale of {param1} is less than {param2}.",
    "{param1} lags behind {param2}.",
    "In terms of size, {param1} is less than {param2}.",
    "{param1} measures less than {param2}.",
    "{param1} is found wanting against {param2}.",
    "{param1} is outmatched by {param2}.",
    "{param1} doesn't exceed {param2}.",
    "{param1} is subdued compared to {param2}.",
    "Relative to {param2}, {param1} is less.",
    "{param1} is at a disadvantage to {param2}.",
    "{param1} falls beneath {param2} in terms of value.",
    "The figure of {param1} is less than {param2}.",
    "{param1} doesn't surpass {param2}.",
    "{param1} is behind {param2} in value.",
    "{param1} yields to {param2}.",
    "{param1} is subpar compared to {param2}.",
    "{param1} is outdone by {param2}.",
    "{param1} is not up to the level of {param2}.",
    "{param1} is undercut by {param2}.",
    "{param1} doesn't match up to {param2}.",
    "{param1} is found lacking compared to {param2}.",
    "{param1} has a lesser value than {param2}."
]

templates_param1_strictely_gt_param2 = [
    "{param1} is strictely greater than {param2}.",
    "{param1} strictely exceeds {param2}.",
    "{param1} is strictely larger than {param2}.",
    "{param1} strictely surpasses {param2}.",
    "{param1} is strictely higher than {param2}.",
    "{param1} is strictely above {param2}.",
    "{param1} is strictely beyond {param2}.",
    "{param1} strictely outclasses {param2}.",
    "{param1} strictely outperforms {param2}.",
    "{param1} is strictelysuperior to {param2}.",
    "{param1} strictely dominates {param2}.",
    "{param1} strictely prevails over {param2}.",
    "{param1} is strictely greater in value than {param2}.",
    "{param1} is strictely more substantial than {param2}.",
    "{param1} strictely stands above {param2}.",
    "{param1} strictely takes precedence over {param2}.",

    "By any measure, {param1} outshines {param2}.",
    "{param1} is unquestionably ahead of the game compared to {param2}.",
    "No doubt about it, {param1} takes the top spot over {param2}.",
    "{param1} boasts an unmistakable edge over {param2}.",
    "{param1} clearly dominates {param2}.",
    "{param1} is in a class of its own, strictly speaking.",
    "{param1} leaves no room for doubt; it's strictly greater than {param2}.",
    "When it comes to superiority, {param1} is in a league of its own compared to {param2}.",
    "{param1} maintains a clear and distinct advantage over {param2}.",
    "Reject any scenario where {param1} equals or falls short of {param2}.",

    "Without any ambiguity, {param1} stands as the undeniable leader over {param2}.",
    "{param1} firmly and irrefutably takes the lead, without question.",
    "{param1} unquestionably outshines {param2} in every aspect.",
    "{param1} is the clear frontrunner, strictly outpacing {param2}.",
    "{param1} sets the gold standard, strictly leaving {param2} behind.",
    "It's crystal clear that {param1} strictly surpasses {param2} in value.",
    "In terms of value, {param1} is in a league of its own, far ahead of {param2}.",
    "{param1} maintains an insurmountable advantage over {param2}, no ifs or buts.",
    "There's no room for debate; {param1} is the unequivocal leader over {param2}.",
    "Without any reservations, {param1} stands as the ultimate leader over {param2}.",
    "{param1} is the clear and undeniable victor, strictly and resoundingly surpassing {param2}.",
    "In no uncertain terms, {param1} is definitively superior to {param2}.",
    "Without any ambiguity, {param1} stands as the undeniable leader over {param2}.",
    "{param1} unquestionably outshines {param2} in every aspect.",
    "{param1} is the clear frontrunner, strictly outpacing {param2}.",
    "{param1} sets the gold standard, strictly leaving {param2} behind.",
    "It's crystal clear that {param1} strictly surpasses {param2} in value.",
    "{param1} takes the lead with absolute certainty, leaving {param2} in the dust.",
    "Without hesitation, {param1} is strictly greater than {param2}.",
    "In terms of value, {param1} is in a league of its own, far ahead of {param2}.",
    "{param1} maintains an insurmountable advantage over {param2}, no ifs or buts.",
    "There's no room for debate; {param1} is the unequivocal leader over {param2}.",
    "Without any reservations, {param1} stands as the ultimate leader over {param2}.",
    "{param1} is the clear and undeniable victor, strictly and resoundingly surpassing {param2}.",
     "{param1} is strictly greater than {param2}.",
    "{param1} is always more than {param2}.",
    "In every case, {param1} exceeds {param2}.",
    "{param1} is invariably higher than {param2}.",
    "Without exception, {param1} is greater than {param2}.",
    "{param1} consistently surpasses {param2}.",
    "There's no instance where {param1} is not greater than {param2}.",
    "{param1} unfailingly exceeds {param2}.",
    "{param1} is undeniably above {param2}.",
    "{param1} is unarguably more than {param2}.",
    "It's a certainty that {param1} is greater than {param2}.",
    "{param1} is decidedly more than {param2}.",
    "{param1} is definitively above {param2}.",
    "{param1} unmistakably surpasses {param2}.",
    "Without doubt, {param1} is greater than {param2}.",
    "It is an absolute that {param1} is higher than {param2}.",
    "{param1} is irrefutably greater than {param2}.",
    "An unequivocal difference with {param1} being greater than {param2}.",
    "{param1} is unconditionally above {param2}.",
    "It's a given that {param1} is more than {param2}.",
    "Undoubtedly, {param1} exceeds {param2}.",
    "{param1} is indisputably higher than {param2}.",
    "It is incontrovertible that {param1} is more than {param2}.",
    "{param1} is clearly above {param2}.",
    "It's irrefutable that {param1} is greater than {param2}.",
    "Evidently, {param1} is more than {param2}.",
    "{param1} is inarguably greater than {param2}.",
    "Manifestly, {param1} is above {param2}.",
    "Patently, {param1} surpasses {param2}.",
    "There's no question that {param1} is greater than {param2}.",
    "{param1} is plainly above {param2}.",
    "It's transparent that {param1} is more than {param2}.",
    "Unequivocally, {param1} exceeds {param2}.",
    "Without controversy, {param1} is above {param2}.",
    "{param1} is palpably more than {param2}.",
    "It's evident that {param1} is greater than {param2}.",
    "Unquestionably, {param1} is more than {param2}.",
    "It's clear that {param1} is above {param2}.",
    "Beyond dispute, {param1} is greater than {param2}.",
    "There's no denying {param1} is above {param2}.",
    "{param1} is unchallengedly greater than {param2}.",
    "It's beyond question that {param1} is more than {param2}.",
    "Assuredly, {param1} is above {param2}.",
    "It goes without saying that {param1} is greater than {param2}.",
    "It is undisputed that {param1} is above {param2}.",
    "{param1} is categorically more than {param2}.",
    "Clearly, {param1} is greater than {param2}.",
    "The superiority of {param1} over {param2} is unquestionable.",
    "{param1} holds a definitive edge over {param2}.",
    "The ascendancy of {param1} over {param2} is clear."

]





templates_greater_than = [
    "{param1} is greater than {param2}",

    "It is required that {param1} exceeds {param2}",
    "The value of {param1} should be higher than {param2}",

    "For this condition, {param1} must be larger than {param2}",
    "In the following case, {param1} is expected to surpass {param2}",
    "Please confirm that {param1} is greater than {param2}",
    "Check if {param1} is greater than {param2} as required",
    "You should have {param1} being greater than {param2}",
    "{param1} should always be larger than {param2}",

    "When comparing {param1} and {param2}, {param1} should dominate",
    "In this scenario, {param1} should be greater than {param2}",

    "The requirement is that {param1} should dominate over {param2}",
    "{param1} should stay ahead of {param2}",
    "When it comes to {param1} and {param2}, {param1} wins",
    "Check that {param1} is always bigger than {param2}",

    "Do not accept any value where {param1} is not greater than {param2}",

    "{param1} is meant to be greater than {param2}",

    "Always check that {param1} prevails over {param2}",

    "{param1} is the winner in the comparison with {param2}",
    "{param1} should keep the lead over {param2}",
    "Confirm that {param1} triumphs over {param2}",

    "{param1} should remain the bigger value compared to {param2}",
    "In any comparison between {param1} and {param2}, {param1} should win",

    "In this scenario, the mandate is that {param1} outshines {param2}",
    "Always ensure that {param1} stays ahead of {param2}",
    "Under all circumstances, {param1} should be the larger value",

    "No exceptions should be made regarding {param1} being greater than {param2}",
    "It is imperative to keep {param1} at a higher level than {param2}",
    "The key requirement is that {param1} takes precedence over {param2}",
    "The golden standard is that {param1} outperforms {param2}",

    "Double-check that {param1} remains superior to {param2}",
    "Ensure that {param1} always stands taller than {param2}",
    "In this case, {param1} must have the upper hand over {param2}",
    "The standard procedure is that {param1} should surpass {param2}",
    "{param1} should consistently outperform {param2}",

    "{param1} should remain the bigger value compared to {param2}",
    "In any comparison between {param1} and {param2}, {param1} should win",

    "In this scenario, the mandate is that {param1} outshines {param2}",
    "Always ensure that {param1} stays ahead of {param2}",
    "Under all circumstances, {param1} should be the larger value",

    "No exceptions should be made regarding {param1} being greater than {param2}",
    "{param1} is greater than {param2}.",
    "{param1} exceeds {param2}.",
    "{param1} is more than {param2}.",
    "{param1} surpasses {param2}.",
    "{param1} is higher than {param2}.",
    "The value of {param1} is above that of {param2}.",
    "{param1} is larger than {param2}.",
    "{param1} outstrips {param2}.",
    "{param1} tops {param2}.",
    "{param1} is superior to {param2}.",
    "{param1} outweighs {param2}.",
    "{param1} overshadows {param2}.",
    "{param1} outdoes {param2}.",
    "{param1} bests {param2}.",
    "{param1} leads {param2}.",
    "{param1} outmatches {param2}.",
    "{param1} outpaces {param2}.",
    "{param1} beats {param2}.",
    "{param1} towers over {param2}.",
    "{param1} outclasses {param2}.",
    "{param1} trumps {param2}.",
    "{param1} outperforms {param2}.",
    "{param1} prevails over {param2}.",
    "{param1} eclipses {param2}.",
    "{param1} outdistances {param2}.",
    "{param1} outguns {param2}.",
    "{param1} outshines {param2}.",
    "{param1} dominates {param2}.",
    "{param1} overshoots {param2}.",
    "{param1} overtops {param2}.",
    "{param1} transcends {param2}.",
    "{param1} outreaches {param2}.",
    "{param1} outsteps {param2}.",
    "{param1} surpasses in value {param2}.",
    "{param1} has the upper hand over {param2}.",
    "{param1} is ahead of {param2}.",
    "{param1} outstrips in magnitude {param2}.",
    "{param1} has an edge over {param2}.",
    "{param1} is superior in terms of {param2}.",
    "{param1} ranks above {param2}.",
    "{param1} holds an advantage over {param2}.",
    "{param1} is preferable to {param2}.",
    "{param1} is more substantial than {param2}.",
    "{param1} has the advantage over {param2}.",
    "{param1} is more significant than {param2}.",
    "{param1} is on a higher level than {param2}.",
    "{param1} takes precedence over {param2}.",
    "{param1} is more considerable than {param2}.",
    "{param1} is more prominent than {param2}.",
    "{param1} is elevated above {param2}."
]
"""


'\ntemplates_all_diff = [\n    "Ensure that {params} are all different.",\n    "No matching values allowed among {params}.",\n    "Keep {params} separate from each other.",\n    "Make sure {params} don\'t overlap.",\n    "{params} should not share any common values.",\n    "Avoid repetitions in {params}.",\n    "None of {params} should be the same.",\n    "{params} must have distinct values.",\n    "All values of {params} must be unique.",\n    "Ensure that there are no duplicates within {params}.",\n    "Prevent any similarities in {params}.",\n    "Disallow any identical values in {params}.",\n    "Insist on distinct values within {params}.",\n    "Prohibit matching values among {params}.",\n    "Guarantee that {params} have no shared values.",\n    "Make it a requirement that {params} are distinct.",\n    "Mandate the uniqueness of values within {params}.",\n    "Enforce distinctness in {params}.",\n    "Impose a restriction on {params} to be all different.",\n    "Enjoin {params} f

In [36]:
# anciennes templates

"""
templates_all_diff = [
    "Ensure that {params} are all different.",
    "No matching values allowed among {params}.",
    "Keep {params} separate from each other.",
    "Make sure {params} don't overlap.",
    "{params} should not share any common values.",
    "Avoid repetitions in {params}.",
    "None of {params} should be the same.",
    "{params} must have distinct values.",
    "All values of {params} must be unique.",
    "Ensure that there are no duplicates within {params}.",
    "Prevent any similarities in {params}.",
    "Disallow any identical values in {params}.",
    "Insist on distinct values within {params}.",
    "Prohibit matching values among {params}.",
    "Guarantee that {params} have no shared values.",
    "Make it a requirement that {params} are distinct.",
    "Mandate the uniqueness of values within {params}.",
    "Enforce distinctness in {params}.",
    "Impose a restriction on {params} to be all different.",
    "Enjoin {params} from sharing any values.",
    "Impel {params} to have no duplicates.",
    "Safeguard the uniqueness of values in {params}.",
    "Assert that {params} have no common values.",
    "Insist upon distinct values within {params}.",
    "Secure the non-repetition of values in {params}.",
    "Command that {params} do not coincide.",
    "Make it a rule that {params} have distinct values.",
    "Oblige {params} to be different from each other.",
    "Require that {params} do not overlap.",
    "Stipulate that {params} should not share values.",
    "Demand the absence of repetitions in {params}.",
    "Specify that {params} must be unique.",
    "State that all values of {params} must be distinct.",
    "Assert that {params} should be all distinct.",
    "Emphasize the distinctness of values within {params}.",
    "Highlight the importance of uniqueness in {params}.",
    "Underline the requirement that {params} have no common values.",
    "Express the necessity for distinct values within {params}.",
    "Communicate the need to prevent repetitions in {params}.",
    "Affirm that {params} must not coincide.",
    "Declare that {params} should have unique values.",
    "Acknowledge the importance of non-overlapping values in {params}.",
    "Admonish against the sharing of values within {params}.",
    "Promote the avoidance of repetitions in {params}.",
    "Reiterate the importance of uniqueness within {params}.",
    "Reinforce the requirement for distinct values in {params}.",
    "Stress the necessity for {params} to be all different.",
]


### ok VERIFIED !!!!

templates_param1_strictely_lt_param2 = [
    "{param1} falls strictely short of {param2}.",
    "{param1} is strictely smaller than {param2}.",
    "{param1} trails clearly behind {param2}.",
    "In terms of value, {param1} is clearly inferior to {param2}.",
    "Without question, {param1} takes the backseat to {param2}.",
    "{param1} is decisively surpassed by {param2}.",
    "{param1} is indisputably beneath {param2}.",
    "There's no doubt about it, {param1} ranks lower than {param2}.",
    "{param1} is emphatically smaller than {param2}.",
    "{param1} is exceedingly lower than {param2}.",
    "When it comes to value, {param1} is exceptionally inferior to {param2}.",
    "{param1} is extremely below {param2}.",
    "{param1} is vastly smaller than {param2}.",
    "It's crystal clear that {param1} strictly falls behind {param2}.",
    "In terms of value, {param1} is decisively inferior to {param2}.",
    "{param1} is markedly lower than {param2} in value.",
    "{param1} falls far short of {param2} consistently.",
    "{param1} consistently trails behind {param2}.",
    "{param1} is consistently outperformed by {param2}.",
    "Without question, {param1} is consistently inferior to {param2}.",
    "{param1} is undeniably and consistently ranked lower than {param2}.",
    "In this comparison, {param1} consistently lags behind {param2}.",
    "Reject any scenario where {param1} equals or surpasses {param2}.",
    "{param1} is not a match for {param2}, strictly speaking.",
    "It's an unwavering truth that {param1} is persistently and strictly less than {param2}.",
    "In all circumstances, {param1} is undeniably and consistently inferior to {param2}.",
    "There's no deviation from the fact that {param1} consistently ranks beneath {param2}.",
    "In every instance, it's consistently proven that {param1} resides in a lower position compared to {param2}.",
    "Without question, {param1} consistently takes a subordinate role compared to {param2}.",
    "{param1} consistently remains in the shadow of {param2}.",
    "In terms of value, {param1} consistently falls far behind {param2}.",
    "{param1} is consistently and decisively inferior to {param2}.",
    "Without any variation, {param1} consistently ranks lower than {param2}.",
    "In each and every context, {param1} consistently resides beneath {param2}.",
    "There's no departure from the fact that {param1} consistently falls short of {param2}.",
    "In every scenario, {param1} is consistently and emphatically smaller than {param2}.",
    "It's consistently and undeniably clear that {param1} is strictly less than {param2}.",
    "In all cases, {param1} is undeniably and consistently inferior to {param2}.",
    "Without fail, {param1} consistently ranks beneath {param2}, no exceptions.",
    "In each and every situation, {param1} consistently remains beneath {param2}.",
    "Without question, {param1} consistently and firmly stays in a position of inferiority compared to {param2}.",
    "It's consistently evident that {param1} takes a subordinate role compared to {param2}.",
    "In the context of value, {param1} consistently lags behind {param2}.",
    "Without any deviations, {param1} consistently falls short of {param2}.",
    "It's an undeniable fact that {param1} is persistently and strictly less than {param2}.",
    "In all instances, {param1} consistently and undeniably ranks lower than {param2}.",
]


templates_param1_strictely_gt_param2 = [
    "{param1} is strictely greater than {param2}.",
    "{param1} exceeds {param2}.",
    "{param1} is larger than {param2}.",
    "{param1} surpasses {param2}.",
    "{param1} is higher than {param2}.",
    "{param1} is above {param2}.",
    "{param1} is beyond {param2}.",
    "{param1} outclasses {param2}.",
    "{param1} outperforms {param2}.",
    "{param1} is superior to {param2}.",
    "{param1} dominates {param2}.",
    "{param1} prevails over {param2}.",
    "{param1} is greater in value than {param2}.",
    "{param1} is more substantial than {param2}.",
    "{param1} stands above {param2}.",
    "{param1} takes precedence over {param2}.",

    "By any measure, {param1} outshines {param2}.",
    "{param1} is unquestionably ahead of the game compared to {param2}.",
    "No doubt about it, {param1} takes the top spot over {param2}.",
    "{param1} boasts an unmistakable edge over {param2}.",
    "{param1} clearly dominates {param2}.",
    "{param1} is in a class of its own, strictly speaking.",
    "{param1} leaves no room for doubt; it's strictly greater than {param2}.",
    "When it comes to superiority, {param1} is in a league of its own compared to {param2}.",
    "{param1} maintains a clear and distinct advantage over {param2}.",
    "Reject any scenario where {param1} equals or falls short of {param2}.",

    "Without any ambiguity, {param1} stands as the undeniable leader over {param2}.",
    "{param1} firmly and irrefutably takes the lead, without question.",
    "{param1} unquestionably outshines {param2} in every aspect.",
    "{param1} is the clear frontrunner, strictly outpacing {param2}.",
    "{param1} sets the gold standard, strictly leaving {param2} behind.",
    "It's crystal clear that {param1} strictly surpasses {param2} in value.",
    "In terms of value, {param1} is in a league of its own, far ahead of {param2}.",
    "{param1} maintains an insurmountable advantage over {param2}, no ifs or buts.",
    "There's no room for debate; {param1} is the unequivocal leader over {param2}.",
    "Without any reservations, {param1} stands as the ultimate leader over {param2}.",
    "{param1} is the clear and undeniable victor, strictly and resoundingly surpassing {param2}.",
    "In no uncertain terms, {param1} is definitively superior to {param2}.",
    "Without any ambiguity, {param1} stands as the undeniable leader over {param2}.",
    "{param1} unquestionably outshines {param2} in every aspect.",
    "{param1} is the clear frontrunner, strictly outpacing {param2}.",
    "{param1} sets the gold standard, strictly leaving {param2} behind.",
    "It's crystal clear that {param1} strictly surpasses {param2} in value.",
    "{param1} takes the lead with absolute certainty, leaving {param2} in the dust.",
    "Without hesitation, {param1} is strictly greater than {param2}.",
    "In terms of value, {param1} is in a league of its own, far ahead of {param2}.",
    "{param1} maintains an insurmountable advantage over {param2}, no ifs or buts.",
    "There's no room for debate; {param1} is the unequivocal leader over {param2}.",
    "Without any reservations, {param1} stands as the ultimate leader over {param2}.",
    "{param1} is the clear and undeniable victor, strictly and resoundingly surpassing {param2}.",
]


# OK VERIFIED  !!!!
abs_val_param1_minus_param_2_diff_param3 = [
    "Ensure that {param3} is not equal to the absolute difference between {param1} and {param2}.",
    "Maintain the condition where {param3} is never the same as the absolute value of the difference between {param1} and {param2}.",
    "Consistently verify that {param3} does not match the absolute value of the difference between {param1} and {param2}.",
    "It's an unbreakable rule that {param3} is never the same as the absolute value of the result when you subtract {param2} from {param1}.",
    "Ensure that {param3} is not the same as the absolute value of the result when you subtract {param2} from {param1}.",
    "In this particular context, the non-negotiable requirement is that {param3} is not equal to the absolute difference between {param2} and {param1}.",
    "Consistently maintain the condition where {param3} is not identical to the absolute value of the result of subtracting {param2} from {param1}.",
    "Reject any value where {param3} is identical to the absolute value of the result of subtracting {param2} from {param1}.",
    "It's an unbreakable rule that {param3} is never the same as the absolute value of the result when you subtract {param1} from {param2}.",
    "Ensure that {param3} is not the same as the absolute value of the result of subtracting {param1} from {param2}.",
    "In this particular context, the non-negotiable requirement is that {param3} is not equal to the absolute difference between {param1} and {param2}.",
    "Consistently verify that {param3} does not match the absolute value of subtracting {param1} from {param2}.",
    "It's an unbreakable rule that {param3} is never the same as the absolute value of the result when you subtract {param2} from {param1}.",
    "Verify that {param3} is not equal to the absolute value of {param1} minus {param2}.",
    "In this particular context, the non-negotiable requirement is that {param3} is not equal to the absolute difference between {param1} and {param2}.",
    "Verify that {param3} is not equal to the absolute value of {param1} minus {param2}.",
    "Verify that the absolute value of {param1} minus {param2} does not equal {param3}.",
    "Ensure that the absolute difference between {param1} and {param2} is not the same as {param3}.",
    "In this context, it's crucial that the absolute value {param1} minus {param2} is not identical to {param3}.",
    "Consistently ensure that the absolute value of {param1} minus {param2} is never equivalent to {param3}.",
    "Reject any scenario where the absolute value of {param1} minus {param2} equals {param3}.",
    "Do not accept the case where the absolute value of result of subtracting {param1} from {param2} is identical to {param3}.",
    "Always ensure that the inequality of the absolute value of the result of subtracting {param1} from {param2} and {param3} is upheld: the two should never be the same.",
    "Verify that the absolute value of the result of subtracting {param1} from {param2} is not equal to {param3}.",
    "Ensure that {param1} and {param2} are not separated by exactly {param3}.",
    "The gap between {param1} and {param2} must not be {param3}.",
    "It's crucial that {param1} and {param2} do not differ by exactly {param3}.",
    "The value {param3} should not represent the span from {param1} to {param2}.",
    "{param1} and {param2} should have a difference other than {param3}.",
    "Make sure the numerical distance between {param1} and {param2} is not {param3}.",
    "The range separating {param1} and {param2} cannot be {param3}.",
    "The measure between {param1} and {param2} should not exactly be {param3}.",
    "Avoid a situation where {param1} is {param3} away from {param2}.",
    "The discrepancy between {param1} and {param2} should not equal {param3}.",
    "Ensure the numerical gap between {param1} and {param2} isn't {param3}.",
    "{param1} and {param2} must not have a {param3} unit disparity.",
    "The separation in value between {param1} and {param2} should not be {param3}.",
    "Keep {param1} and {param2} from having a {param3} value difference.",
    "The distance measured from {param1} to {param2} must not total {param3}.",

]


# VERIFIED OK !!!!!!!!

abs_val_p1_minus_p2_equal_abs_val_p3_minus_p4=[
    "Confirm that the absolute difference between {param1} and {param2} equals the absolute difference between {param3} and {param4}.",
    "Ensure that the absolute value of the result when you subtract {param2} from {param1} is the same as when you subtract {param4} from {param3}.",
    "In this scenario, the requirement is that the absolute difference between {param1} and {param2} is identical to the absolute difference between {param3} and {param4}.",
    "Always ensure that the result of subtracting {param2} from {param1} has the same absolute value as the result of subtracting {param4} from {param3}.",
    "Do not accept any value where the absolute difference between {param1} and {param2} is not equal to the absolute difference between {param3} and {param4}.",
    "Maintain the equality of absolute differences: the absolute difference between {param1} and {param2} equals the absolute difference between {param3} and {param4}.",
    "Always verify that the absolute value of the difference between {param1} and {param2} is equal to the absolute value of the difference between {param3} and {param4}.",
    "The top priority is that the absolute difference between {param1} and {param2} consistently matches the absolute difference between {param3} and {param4}.",
    "In this scenario, the requirement is that the absolute difference between {param2} and {param1} is identical to the absolute difference between {param4} and {param3}.",
    "Always ensure that the result of subtracting {param1} from {param2} has the same absolute value as the result of subtracting {param3} from {param4}.",
    "Do not accept any value where the absolute difference between {param2} and {param1} is not equal to the absolute difference between {param4} and {param3}.",
    "Maintain the equality of absolute differences: the absolute difference between {param2} and {param1} equals the absolute difference between {param4} and {param3}.",
    "Always verify that the absolute value of the difference between {param2} and {param1} is equal to the absolute value of the difference between {param4} and {param3}.",
    "The top priority is that the absolute difference between {param2} and {param1} consistently matches the absolute difference between {param4} and {param3}.",
    "Consistently verify that the absolute value of the difference between {param1} and {param2} is the same as the absolute value of the difference between {param3} and {param4}.",
    "It's absolutely certain that the absolute value of the result of subtracting {param2} from {param1} is always the same as the absolute value of the result of subtracting {param4} from {param3}.",
    "Ensure that the absolute value of the result when you subtract {param4} from {param3} equals the absolute value of the result when you subtract {param2} from {param1}.",
    "In this scenario, the requirement is that the absolute difference between {param4} and {param3} is the same as the absolute difference between {param2} and {param1}.",
    "Always ensure that the result of subtracting {param4} from {param3} has the same absolute value as the result of subtracting {param2} from {param1}.",
    "Do not accept any value where the absolute difference between {param4} and {param3} is not equal to the absolute difference between {param2} and {param1}.",
    "Maintain the equality of absolute differences: the absolute difference between {param4} and {param3} equals the absolute difference between {param2} and {param1}.",
    "Always verify that the absolute value of the difference between {param4} and {param3} is equal to the absolute value of the difference between {param2} and {param1}.",
    "The top priority is that the absolute difference between {param4} and {param3} consistently matches the absolute difference between {param2} and {param1}.",
    "It's non-negotiable that the absolute value of the result of subtracting {param3} from {param4} is always the same as the absolute value of the result of subtracting {param2} from {param1}.",
    "Affirm that the absolute difference between {param3} and {param4} matches the absolute difference between {param1} and {param2}.",
    "The gap between {param1} and {param2} should equal the gap between {param3} and {param4}.",
    "Verify that the span from {param1} to {param2} is the same as from {param3} to {param4}.",
    "The length separating {param1} and {param2} must be identical to that between {param3} and {param4}.",
    "Make sure the stretch from {param1} to {param2} equates to the stretch from {param3} to {param4}.",
    "Confirm the separation between {param1} and {param2} equals the separation between {param3} and {param4}.",
    "The interval between {param1} and {param2} needs to match the interval between {param3} and {param4}.",
    "The range spanning from {param1} to {param2} should be the same as from {param3} to {param4}.",
    "Ensure that the measure of distance between {param1} and {param2} is the same as between {param3} and {param4}.",
    "The span between {param1} and {param2} must equate to the span between {param3} and {param4}.",
    "Make sure the interval distance between {param1} and {param2} is the same as between {param3} and {param4}.",
    "Ensure that the stretch length from {param1} to {param2} equals that from {param3} to {param4}.",
    "The span length between {param1} and {param2} needs to be identical to that between {param3} and {param4}.",
    "Verify the length measure between {param1} and {param2} matches that between {param3} and {param4}.",
    "The distance span from {param1} to {param2} should be equivalent to that from {param3} to {param4}.",
    "Make certain the separation measure between {param1} and {param2} equals that between {param3} and {param4}.",
    "Verify that the range difference between {param1} and {param2} is equal to that between {param3} and {param4}.",
    "The distance ratio from {param1} to {param2} should be the same as from {param3} to {param4}.",
    "Ensure the numerical separation from {param1} to {param2} equates to that from {param3} to {param4}.",
    "The gap measurement between {param1} and {param2} must align with the measurement between {param3} and {param4}.",
    "Confirm that the length discrepancy between {param1} and {param2} matches the discrepancy between {param3} and {param4}.",
    "The space between {param1} and {param2} should equal the space between {param3} and {param4}.",
    "Make sure the interval gap from {param1} to {param2} is identical to the gap from {param3} to {param4}.",
    "Verify the range span between {param1} and {param2} equals the span between {param3} and {param4}.",
    "Ensure the distance between {param1} and {param2} matches the distance between {param3} and {param4}.",
    "The separation distance between {param1} and {param2} should be the same as between {param3} and {param4}.",
    "Confirm that the length range between {param1} and {param2} equates to the range between {param3} and {param4}.",
    "The interval range from {param1} to {param2} must be equal to the range from {param3} to {param4}.",
    "Ensure the length separation between {param1} and {param2} matches the separation between {param3} and {param4}.",
    "The numerical stretch between {param1} and {param2} should be the same as between {param3} and {param4}.",
]


abs_val_p1_minus_p2_equal_p_3 = [
    "Ensure that the absolute difference between {param1} and {param2} equals {param3}.",
    "The condition is that the magnitude of the difference between {param1} and {param2} should be {param3}.",
    "Verify that the absolute value of ({param1} subtracted from {param2}) is {param3}.",
    "Make sure that the absolute difference between {param1} and {param2} matches {param3}.",
    "Maintain the absolute difference between {param1} and {param2} as {param3} at all times.",
    "It's non-negotiable that the absolute value of the result of subtracting {param1} from {param2} matches {param3}.",
    "Ensure that the absolute difference between {param1} and {param2} remains {param3}.",
    "The cardinal rule is that the absolute difference between {param1} and {param2} must be {param3}.",
    "Do not allow the result of subtracting {param1} from {param2} in absolute value to be unequal to {param3} under any circumstances.",
    "The unbreakable rule is that the absolute difference between {param1} and {param2} cannot deviate from {param3}."
    "Ensure that the absolute difference between {param1} and {param2} is equivalent to {param3}.",
    "Verify that the absolute value of the subtraction of {param2} from {param1} equals {param3}.",
    "Always ensure that the absolute value of the result of {param2} subtracted from {param1} is not different from {param3}.",
    "The cardinal rule is that the absolute difference between {param2} and {param1} must be {param3}.",
    "Do not allow the result of {param2} subtracted from {param1} in absolute value to be unequal to {param3} under any circumstances.",
    "The unbreakable rule is that the absolute difference between {param2} and {param1} cannot deviate from {param3}.",
    "Make certain that the absolute value of ({param1} with {param2} subtracted) is equal to {param3}.",
    "Ensure that the absolute value of the difference between {param1} and {param2} is the same as {param3}.",
    "Make sure that the absolute difference between {param1} and {param2} matches {param3}.",
    "Always ensure that the absolute value of the result of {param1} subtracted from {param2} is not different from {param3}.",
    "Do not allow the result of {param1} subtracted from {param2} in absolute value to be unequal to {param3} under any circumstances.",
    "The unbreakable rule is that the absolute difference between {param1} and {param2} cannot deviate from {param3}.",
    "Confirm that the absolute value of the difference between {param1} and {param2} precisely equals {param3}.",
    "Double-check that the absolute value of the subtraction of {param2} from {param1} is {param3}.",
    "Ensure that the absolute difference between {param2} and {param1} consistently matches {param3}.",
    "The unalterable rule is that the absolute difference between {param2} and {param1} cannot deviate from {param3}.",
    "Always ensure that the absolute value of the result of {param2} subtracted from {param1} is not different from {param3}.",
    "Maintain the absolute difference between {param2} and {param1} as {param3} without exception.",
    "It's non-negotiable that the absolute value of the result of {param2} subtracted from {param1} matches {param3}.",
    "Ensure that the absolute difference between {param2} and {param1} always coincides with {param3}."
    "Ensure that {param3} is exactly the distance between {param1} and {param2}.",
    "The value of {param3} must match the distance between {param1} and {param2}.",
    "It is required that {param3} equals the distance separating {param1} and {param2}.",
    "Make sure {param3} reflects the exact distance between {param1} and {param2}.",
    "{param3} should equal the distance between {param1} and {param2}.",
    "The amount {param3} must be the distance from {param1} to {param2}.",
    "{param3} needs to represent the distance separating {param1} and {param2}.",
    "The figure {param3} should match the distance between {param1} and {param2}.",
    "Ensure the distance between {param1} and {param2} is exactly {param3}.",
    "The distance measure between {param1} and {param2} should be {param3}.",
    "Ensure the gap measured between {param1} and {param2} is {param3}.",
    "{param3} has to be the exact span between {param1} and {param2}.",
    "The length between {param1} and {param2} must equal {param3}.",
    "Ensure {param3} represents the distance between {param1} and {param2}.",
    "The separation between {param1} and {param2} should be {param3}.",
    "The distance covered from {param1} to {param2} must be {param3}.",
    "{param3} should be the length spanning between {param1} and {param2}.",
    "Make sure the separation distance between {param1} and {param2} equals {param3}.",
    "The interval between {param1} and {param2} must equate to {param3}.",
    "The reach between {param1} and {param2} needs to be exactly {param3}.",
    "Set the distance from {param1} to {param2} to exactly {param3}.",
    "Ensure the span between {param1} and {param2} matches {param3}.",
    "The distance metric between {param1} and {param2} should be set to {param3}.",
    "Confirm that the interval between {param1} and {param2} is {param3}.",
    "Match the distance between {param1} and {param2} to {param3}.",
    "The range between {param1} and {param2} should precisely be {param3}.",
    "Ensure the length spanned between {param1} and {param2} is {param3}.",
    "{param3} must be the reach from {param1} to {param2}.",
    "The distance value between {param1} and {param2} must be set as {param3}.",

    ]

templates_sum_x1_x2_diff_x3 = [
    "Keep {param1} and {param2} distinct from {param3} at all times.",
    "Check that {param1} increased by {param2} is distinct from {param3}.",
    "Verify that the sum of {param1} and {param2} is never equal to {param3}.",
    "Under no circumstances should {param1} plus {param2} match {param3}.",
    "The requirement is that {param1} plus {param2} should consistently differ from {param3}.",
    "Double-check that {param1} added to {param2} does not equal {param3}.",
    "Confirm that {param1} added to {param2} is always different from {param3}.",
    "In this scenario, {param1} aggregate with {param2} should not equal {param3}.",
    "Always ensure that the sum of {param1} and {param2} are distinct from {param3}.",
    "The primary condition is that {param1} plus {param2} cannot be the same as {param3}.",
    "Do not accept any value where {param1} plus {param2} equals {param3}.",
    "Always ensure that the sum of {param1} and {param2} is different from {param3}.",
    "Under all circumstances, {param1} added to {param2} should not be the same as {param3}.",
    "No exceptions should be made regarding {param1} augmented by {param2} and {param3} being different.",
    "It is imperative that {param1} augmented by {param2} and {param3} are not the same.",
    "The golden standard is that {param1} augmented by {param2} and {param3} are distinct.",
    "Never allow {param1} increased by {param2} to match {param3}.",
    "Double-check that {param1} augmented by {param2} and {param3} are distinct.",
    "Ensure that {param1} augmented by {param2} does not equal {param3}.",
    "Always maintain that {param1} augmented by {param2} and {param3} are different values.",
    "Keep {param1} augmented by {param2} distinct from {param3} at all times.",
    "In this scenario, {param1} augmented by {param2} and {param3} should be different.",
    "Ensure that {param1} added to {param2} is never equal to {param3}.",
    "The mandate is that {param1} added to {param2} must not match {param3}.",
    "Check that {param1} and {param2} produce a sum distinct from {param3}.",
    "Always double-check that {param1} added to {param2} doesn't equal {param3}.",
    "Make sure that {param1} added to {param2} is always different from {param3}.",
    "The rule is that {param1} added to {param2} should never result in {param3}.",
    "When adding {param1} and {param2}, they should not equal {param3}.",
    "Verify that {param1} added to {param2} doesn't match {param3}.",
    "The key requirement is that {param1} added to {param2} and {param3} are distinct.",
    "Never deviate from {param1} added to {param2} not matching {param3}.",
    "Make sure that {param1} added to {param2} consistently does not match {param3}.",
    "Under no circumstances should {param3} equal {param1} added to {param2}.",
    "Check that {param3} is always different from {param1} joined with {param2}.",
    "Always maintain that {param3} and {param1} joined with {param2} are distinct.",
    "Ensure that {param3} does not equal {param1} joined with {param2}.",
    "It's non-negotiable that {param1} joined with {param2} is distinct from {param3}.",
    "For {param1} joined with {param2} and {param3}, the rule is that they are not the same.",
    "Never allow {param1} brought together with {param2} to match {param3}.",
    "Double-check that {param3} and {param1} brought together with {param2} are distinct.",
    "Ensure that {param1} brought together with {param2} does not equal {param3}.",
    "Always maintain that {param3} and {param1} brought together with {param2} are different values.",
    "Keep {param3} distinct from {param1} summed with {param2} at all times.",
    "Ensure that {param3} does not equal {param1} Put together with {param2}.",
    "It's non-negotiable that {param1} Put together with {param2} is distinct from {param3}.",
    "For {param1} Put together with {param2} and {param3}, the rule is that they are not the same.",
    "Never allow {param1} Put together with {param2} to match {param3}.",
    "Double-check that {param3} and {param1} Put together with {param2} are distinct.",
    "Ensure that {param1} aggregate with {param2} does not equal {param3}.",
    "Always maintain that {param3} and {param1} aggregate with {param2} are different values.",
    "Keep {param3} distinct from {param1} aggregate with {param2} at all times.",
    "The cardinal rule is that {param1} and {param2} must never add up to {param3}.",
    "Do not allow {param1} plus {param2} to equal {param3} under any circumstances.",
    "Make it a priority that {param1} increased by {param2} do not result in {param3}.",
    "Triple-check that {param1} plus {param2} is consistently distinct from {param3}.",
    "At no point should {param3} be equal to the sum of {param1} and {param2}.",
    "Ensure that {param3} and the sum of {param1} and {param2} remain distinct.",
    "The unbreakable rule is that {param1} plus {param2} cannot be equal to {param3}.",
    "Guard against {param1} and {param2} adding up to {param3} at all times.",
    "Maintain a strict separation between {param1} plus {param2} and {param3}.",
    "It's crucial that {param1} and {param2} do not combine to equal {param3}.",
    "In every calculation, make sure {param1} plus {param2} does not match {param3}.",
    "Under no circumstances should {param1} increased by {param2} equal {param3}.",
]


templates_equal_constraint = [
    "{param1} is equal to {param2}",
    "{param1} and {param2} are the same",
    "It is required that {param1} equals {param2}",
    "{param1} should match {param2}",
    "{param2} must be identical to {param1}",
    "In this scenario, {param1} and {param2} are equal",
    "Please confirm that {param1} equals {param2}",
    "Make sure {param1} is identical to {param2}",
    "If you see {param1}, it should mean {param2} is the same",
    "No exceptions allowed: {param1} equals {param2}",
    "{param1} and {param2} have to be the same",
    "{param1} and {param2} must be identical",
    "To satisfy the condition, {param1} should match {param2}",
    "{param2} should always be the same as {param1}",
    "{param1} and {param2} should match",
    "{param2} is required to be identical to {param1}",
    "{param1} should consistently match {param2}",
    "Verify that {param1} equals {param2}",
    "Always ensure that {param2} and {param1} are the same",
    "{param1} is intended to be equal to {param2}",
    "{param2} must always match {param1}",
    "Always check that {param1} and {param2} are the same",
    "Ensure that {param1} and {param2} are always equal",
    "{param1} and {param2} emerge as equal values",
    "{param1} should always maintain equality with {param2}",
    "Confirm that {param1} and {param2} match",
    "The expectation is that {param2} is equal to {param1}",
    "Maintain {param2} as an equal value to {param1}",
    "It's mandatory for {param1} and {param2} to match",
    "{param1} and {param2} should always be equal",
    "Never compromise on {param1} matching {param2}",
    "In this scenario, {param1} and {param2} must remain equal",
    "The golden principle is that {param1} equals {param2}",
    "Never deviate from {param2} matching {param1}",
    "Make sure that {param1} consistently matches {param2}",
    "Under no circumstances should {param2} not be the same as {param1}",
    "Check that {param2} and {param1} maintain equality",
    "Under all circumstances, {param2} should be the same as {param1}",
    "The primary condition is that {param1} and {param2} match",
    "No exceptions should be made regarding {param1} and {param2} being equal",
    "It is imperative that {param1} and {param2} match",
    "The key requirement is that {param1} and {param2} are the same",
    "The golden standard is that {param1} and {param2} match",
    "Never allow {param2} to not match {param1}",
    "Double-check that {param1} and {param2} match",
    "Ensure that {param2} equals {param1}",
    "Always maintain that {param2} and {param1} are equal values",
    "Keep {param2} equal to {param1} at all times",
    "In this scenario, {param2} and {param1} should match",
    "Ensure that {param1} matches {param2}",
    "The requirement is that {param1} and {param2} match",
    "Maintain {param1} and {param2} as equal values",
    "Confirm that {param1} and {param2} are the same",
    "The golden rule is that {param1} and {param2} match",
    "Under no circumstances should {param1} not equal {param2}",
    "Double-check that {param2} is the same as {param1}",
    "Ensure {param2} matches {param1}",
    "Confirm that {param1} and {param2} match",
    "The expectation is that {param2} is equal to {param1}",
    "Maintain {param2} as an equal value to {param1}",
    "It's mandatory for {param1} and {param2} to match",
    "{param1} and {param2} should always be equal",
    "Never compromise on {param1} matching {param2}",
    "In this scenario, {param1} and {param2} must remain equal",
]

templates_different_than = [
    "{param1} is different from {param2}",
    "{param2} is not the same as {param1}",
    "{param1} and {param2} are distinct",
    "It is required that {param1} and {param2} are not equal",
    "{param1} should not match {param2}",
    "{param2} must not be identical to {param1}",
    "In this scenario, {param1} and {param2} are different",
    "Please confirm that {param1} does not equal {param2}",
    "Check if {param1} is not the same as {param2}",
    "You should have {param1} distinct from {param2}",
    "{param1} should always differ from {param2}",
    "The rule is that {param1} cannot be equal to {param2}",
    "When comparing {param1} and {param2}, they should not match",
    "Make sure {param1} is not identical to {param2}",
    "If you see {param1}, it should mean {param2} is not the same",
    "No exceptions allowed: {param1} different than {param2}",
    "{param1} and {param2} have to be distinct",
    "{param1} and {param2} must not be identical",
    "To satisfy the condition, {param1} should not equal {param2}",
    "In this scenario, {param1} and {param2} should be different",
    "Ensure that {param2} does not equal {param1}",
    "The requirement is that {param1} and {param2} should differ",
    "{param1} should stay distinct from {param2}",
    "When it comes to {param1} and {param2}, they should not match",
    "Check that {param1} and {param2} do not equal",
    "The primary condition is that {param2} cannot be equal to {param1}",
    "Do not accept any value where {param1} and {param2} match",
    "Maintain {param1} and {param2} as distinct values",
    "Confirm that {param1} and {param2} are not the same",
    "The golden rule is that {param1} and {param2} are different",
    "Under no circumstances should {param1} equal {param2}",
    "Double-check that {param2} is not the same as {param1}",
    "Ensure {param2} is different from {param1}",
    "In this case, {param2} should not equal {param1}",
    "{param1} should consistently differ from {param2}",
    "Verify that {param1} does not equal {param2}",
    "Always ensure that {param2} and {param1} are not the same",
    "{param1} is intended to be different from {param2}",
    "{param2} must never match {param1}",
    "The top priority is that {param2} differs from {param1}",
    "It's non-negotiable that {param1} does not equal {param2}",
    "For {param1} and {param2}, the rule is that they are different",
    "{param2} should always remain different from {param1}",
    "The key requirement is that {param1} and {param2} are not the same",
    "The golden standard is that {param1} and {param2} are distinct",
    "Never allow {param2} to match {param1}",
    "Double-check that {param1} and {param2} are distinct",
    "Ensure that {param2} does not equal {param1}",
    "Always maintain that {param2} and {param1} are different values",
    "Keep {param2} distinct from {param1} at all times",
    "In this scenario, {param2} and {param1} should be different",
    "Ensure that {param1} does not match {param2}",
    "The requirement is that {param2} and {param1} should differ",
    "{param2} should never be the same as {param1}",
    "{param1} and {param2} should not match",
    "{param2} is not allowed to be equal to {param1}",
    "Check that {param1} and {param2} do not equal",
    "The primary condition is that {param2} cannot be equal to {param1}",
    "Do not accept any value where {param1} and {param2} match",
    "Maintain {param1} and {param2} as distinct values",
    "Confirm that {param1} and {param2} are not the same",
	  "{param2} should always remain different from {param1}",
]

templates_less_than = [
    "{param1} is less than {param2}",
    "{param2} cannot be less than {param1}",
    "It is required that {param1} is less than {param2}",
    "The value of {param1} should be lower than {param2}",
    "Ensure that {param2} is greater than {param1}",
    "For this condition, {param2} must be larger than {param1}",
    "In the following case, {param2} is expected to exceed {param1}",
    "Please confirm that {param1} is less than {param2}",
    "Check if {param1} is less than {param2} as required",
    "You should have {param1} being smaller than {param2}",
    "{param1} should always be lesser than {param2}",
    "{param2} is not allowed to be less than {param1}",
    "Always maintain {param1} as the smaller value under {param2}",
    "Keep {param1} lower than {param2} at all times",
    "In this scenario, {param1} should be lesser than {param2}",
    "Ensure that {param2} remains greater than {param1}",
    "The requirement is that {param1} should be inferior to {param2}",
    "{param1} should stay behind {param2}",
    "When it comes to {param1} and {param2}, {param2} wins",
    "Check that {param1} is always smaller than {param2}",
    "The primary condition is that {param2} should be greater than {param1}",
    "Do not accept any value where {param1} is not less than {param2}",
    "Maintain {param1} at a lower level than {param2}",
    "Confirm that {param1} takes a back seat to {param2}",
    "The golden rule is that {param2} outshines {param1}",
    "Under no circumstances should {param1} exceed {param2}",
    "Double-check that {param2} always surpasses {param1}",
    "Ensure {param2} stands taller than {param1}",
    "In this case, {param2} must have the upper hand over {param1}",
    "The standard is that {param2} should surpass {param1}",
    "{param1} should consistently fall short of {param2}",
    "Verify that {param1} never surpasses {param2}",
    "Always ensure that {param2} does not fall short of {param1}",
    "{param1} is intended to be less than {param2}",
    "{param2} must always overshadow {param1}",
    "Always check that {param2} is the dominant value over {param1}",

    "{param2} emerges as the winner in the comparison with {param1}",
    "{param2} should always maintain the lead over {param1}",
    "Confirm that {param2} prevails over {param1}",

    "Maintain {param2} as the top value over {param1}",
    "It's mandatory for {param2} to dominate {param1}",
    "{param2} should always have the upper hand over {param1}",
    "Never compromise on {param2} being greater than {param1}",
    "In this scenario, {param2} must maintain its superiority over {param1}",
    "Always ensure that {param2} stays ahead of {param1}",
    "Under all circumstances, {param2} should be the larger value",
    "The primary condition is that {param1} should be less than {param2}",
    "No exceptions should be made regarding {param2} being greater than {param1}",
    "It is imperative to keep {param2} at a higher level than {param1}",
    "{param2} should consistently outperform {param1}",
    "Verify that {param1} never surpasses {param2}",
    "Always ensure that {param1} does not outperform {param2}",
    "The golden principle is that {param2} exceeds {param1}",
    "Never deviate from {param2} being the larger value over {param1}",
    "Make sure that {param2} consistently trumps {param1}",
    "Under no circumstances should {param1} take the lead over {param2}",
    "Check that {param2} maintains its supremacy over {param1}",
    "Always verify that {param2} prevails over {param1}",
    "The top priority is that {param2} is greater than {param1}",
    "It's non-negotiable that {param1} is less than {param2}",
    "For {param1} and {param2}, the rule is that {param2} wins",
    "{param2} should always remain the bigger value compared to {param1}",
    "In any comparison between {param1} and {param2}, {param2} should win",
    "Double-check that {param1} never surpasses {param2}",
    "In this scenario, the mandate is that {param2} outshines {param1}",
    "Always ensure that {param2} stays ahead of {param1}",
    "Under all circumstances, {param2} should be the larger value",
    "No exceptions should be made regarding {param2} being the dominant value",
]


templates_greater_than = [
    "{param1} is greater than {param2}",
    "{param2} cannot be greater than {param1}",
    "It is required that {param1} exceeds {param2}",
    "The value of {param1} should be higher than {param2}",
    "Ensure that {param2} is less than {param1}",
    "For this condition, {param1} must be larger than {param2}",
    "In the following case, {param1} is expected to surpass {param2}",
    "Please confirm that {param1} is greater than {param2}",
    "Check if {param1} is greater than {param2} as required",
    "You should have {param1} being greater than {param2}",
    "{param1} should always be larger than {param2}",
    "The rule is that {param2} cannot be more than {param1}",
    "When comparing {param1} and {param2}, {param1} should dominate",
    "In this scenario, {param1} should be greater than {param2}",
    "Ensure that {param2} remains less than {param1}",
    "The requirement is that {param1} should dominate over {param2}",
    "{param1} should stay ahead of {param2}",
    "When it comes to {param1} and {param2}, {param1} wins",
    "Check that {param1} is always bigger than {param2}",
    "The primary condition is that {param2} is less than {param1}",
    "Do not accept any value where {param1} is not greater than {param2}",
    "Verify that {param2} doesn't rise above {param1}",
    "Never allow {param2} to outperform {param1}",
    "{param1} is meant to be greater than {param2}",
    "{param2} must never overshadow {param1}",
    "Always check that {param1} prevails over {param2}",
    "Ensure that {param2} is always trailing behind {param1}",
    "{param1} is the winner in the comparison with {param2}",
    "{param1} should keep the lead over {param2}",
    "Confirm that {param1} triumphs over {param2}",
    "It's non-negotiable that {param2} is less than {param1}",
    "For {param1} and {param2}, the rule is that {param1} wins",
    "{param1} should remain the bigger value compared to {param2}",
    "In any comparison between {param1} and {param2}, {param1} should win",
    "Double-check that {param2} never surpasses {param1}",
    "In this scenario, the mandate is that {param1} outshines {param2}",
    "Always ensure that {param1} stays ahead of {param2}",
    "Under all circumstances, {param1} should be the larger value",
    "The primary condition is that {param2} should be less than {param1}",
    "No exceptions should be made regarding {param1} being greater than {param2}",
    "It is imperative to keep {param1} at a higher level than {param2}",
    "The key requirement is that {param1} takes precedence over {param2}",
    "The golden standard is that {param1} outperforms {param2}",
    "Never allow {param2} to exceed {param1}",
    "Double-check that {param1} remains superior to {param2}",
    "Ensure that {param1} always stands taller than {param2}",
    "In this case, {param1} must have the upper hand over {param2}",
    "The standard procedure is that {param1} should surpass {param2}",
    "{param1} should consistently outperform {param2}",
    "Verify that {param2} never surpasses {param1}",
    "{param1} should remain the bigger value compared to {param2}",
    "In any comparison between {param1} and {param2}, {param1} should win",
    "Double-check that {param2} never surpasses {param1}",
    "In this scenario, the mandate is that {param1} outshines {param2}",
    "Always ensure that {param1} stays ahead of {param2}",
    "Under all circumstances, {param1} should be the larger value",
    "The primary condition is that {param2} should be less than {param1}",
    "No exceptions should be made regarding {param1} being greater than {param2}"
]


abs_val_p1_minus_p2_diff_abs_val_p3_minus_p4=[

    "Ensure that the gap between {param1} and {param2} does not match the gap between {param3} and {param4}.",
    "It is required that the distance from {param1} to {param2} is not the same as from {param3} to {param4}.",
    "The separation between {param1} and {param2} should not equate to that between {param3} and {param4}.",
    "The disparity in values between {param1} and {param2} must differ from that between {param3} and {param4}.",
    "Confirm that the span from {param1} to {param2} doesn't mirror the span from {param3} to {param4}.",
    "The interval separating {param1} and {param2} cannot be identical to the interval between {param3} and {param4}.",
    "The difference in magnitude between {param1} and {param2} should not reflect the difference between {param3} and {param4}.",
    "Ensure the numeric gap between {param1} and {param2} isn't the same as between {param3} and {param4}.",
    "The range from {param1} to {param2} should not match the range from {param3} to {param4}.",
    "Confirm that the numerical distance between {param1} and {param2} differs from {param3} to {param4}.",
    "Ensure {param1} and {param2} are separated by a different measure than {param3} and {param4}.",
    "The length from {param1} to {param2} must not equal the length from {param3} to {param4}.",
    "Verify that the offset between {param1} and {param2} is unlike the offset between {param3} and {param4}.",
    "It is crucial that the deviation between {param1} and {param2} is not mirrored by {param3} and {param4}.",
    "Ascertain that the span between {param1} and {param2} is unique from the span between {param3} and {param4}.",
    "Ensure the distance measure between {param1} and {param2} isn't equivalent to {param3} and {param4}.",
    "The gap from {param1} to {param2} should not be identical to the gap from {param3} to {param4}.",
    "Confirm the disparity between {param1} and {param2} doesn't align with the disparity between {param3} and {param4}.",
    "The numerical distance between {param1} and {param2} must be different from that between {param3} and {param4}.",
    "The difference in position between {param1} and {param2} shouldn't equate to {param3} and {param4}.",
    "Ensure the stretch between {param1} and {param2} is not the same as the stretch between {param3} and {param4}.",
    "Verify the dissimilarity in space between {param1} and {param2} against {param3} and {param4}.",
    "Make certain the leap from {param1} to {param2} doesn't replicate the leap from {param3} to {param4}.",
    "Ascertain that the stride between {param1} and {param2} is not akin to the stride between {param3} and {param4}.",
    "The breach between {param1} and {param2} should not resemble the breach between {param3} and {param4}.",
    "The rift between {param1} and {param2} must be unlike the rift between {param3} and {param4}.",
    "The distance in terms of units between {param1} and {param2} should not align with {param3} and {param4}.",
    "Ensure that the span in value between {param1} and {param2} is distinct from {param3} to {param4}.",
    "The disparity in length between {param1} and {param2} must not be similar to {param3} and {param4}.",
    "Confirm that the space between {param1} and {param2} does not equate the space between {param3} and {param4}.",
    "It is necessary that the stretch in figures between {param1} and {param2} is different from {param3} to {param4}.",
    "Verify the variance in distance between {param1} and {param2} against that between {param3} and {param4}.",
    "Make sure the difference in extent between {param1} and {param2} isn't the same as {param3} and {param4}.",
    "Ascertain that the interval length between {param1} and {param2} is not comparable to {param3} and {param4}.",
    "The physical separation between {param1} and {param2} should not match the separation between {param3} and {param4}.",
    "Ensure that the absolute difference between {param1} and {param2} is not the same as between {param3} and {param4}.",
    "The absolute difference of {param1} and {param2} should not equal the absolute difference of {param3} and {param4}.",
    "The absolute variance between {param1} and {param2} must differ from the absolute variance between {param3} and {param4}.",
    "Confirm that the absolute deviation of {param1} from {param2} isn't the same as {param3} from {param4}.",
    "Ensure the absolute separation between {param1} and {param2} is not akin to that between {param3} and {param4}.",
    "The absolute difference in value between {param1} and {param2} should not replicate {param3} and {param4}.",
    "Verify that the absolute spread between {param1} and {param2} is unique from the spread between {param3} and {param4}.",
    "Ascertain that the absolute range from {param1} to {param2} isn't equal to the range from {param3} to {param4}.",
    "The absolute breach between {param1} and {param2} must not mirror the breach between {param3} and {param4}.",
    "Confirm the absolute rift between {param1} and {param2} does not match the rift between {param3} and {param4}.",
    "Ensure the absolute gap measure between {param1} and {param2} isn't equivalent to {param3} and {param4}.",
    "The absolute length difference between {param1} and {param2} should not equate to {param3} and {param4}.",
    "Verify that the absolute offset between {param1} and {param2} differs from {param3} and {param4}.",
    "Confirm that the absolute span in length between {param1} and {param2} is not comparable to {param3} and {param4}."
]


sum_params_equal_param = [
    "Ensure the total of {params} equals {param}.",
    "The sum of {params} must be exactly {param}.",
    "Verify that adding {params} together results in {param}.",
    "It is required that {params} combined equal {param}.",
    "Confirm that the aggregate of {params} is {param}.",
    "The collective amount of {params} should match {param}.",
    "Ensure that the cumulative total of {params} amounts to {param}.",
    "The total sum of {params} needs to be precisely {param}.",
    "Verify the combined value of {params} equals {param}.",
    "Ascertain that summing up {params} yields {param}.",
    "The overall total of {params} must equate to {param}.",
    "Confirm that the joint sum of {params} is exactly {param}.",
    "Ensure the accumulated amount of {params} is {param}.",
    "The collective sum of {params} should be {param}.",
    "Verify the total addition of {params} equals {param}.",
    "It's essential that {params} together total {param}.",
    "Confirm the combined sum of {params} reaches {param}.",
    "The entire sum of {params} must tally up to {param}.",
    "Ensure the total of all {params} is precisely {param}.",
    "The grand total of {params} should equal {param}.",
    "Verify the sum total of {params} is {param}.",
    "Ascertain the full total of {params} equals {param}.",
    "The complete sum of {params} must be {param}.",
    "Confirm that the overall addition of {params} is {param}.",
    "Ensure the sum calculation of {params} matches {param}.",
    "The additive total of {params} should equate to {param}.",
    "Verify the entire aggregation of {params} equals {param}.",
    "It's necessary that the full addition of {params} is {param}.",
    "Confirm the collective calculation of {params} totals {param}.",
    "{param} must be the total of {params}.",
    "{param} should equal the sum of {params}.",
    "The value of {param} is the collective sum of {params}.",
    "{param} is required to match the total of {params}.",
    "The exact amount of {param} should be the sum of {params}.",
    "{param} needs to be the combined total of {params}.",
    "The figure {param} must match the aggregate of {params}.",
    "{param} should represent the cumulative sum of {params}.",
    "For {param}, the sum of {params} is essential.",
    "Equating {param} with the total of {params} is necessary.",
    "The amount {param} should be the result of adding {params}.",
    "{param} is the required total of {params}.",
    "The sum of {params} should precisely equal {param}.",
    "{param} must be the end result of summing {params}.",
    "Aim for {param} to be the collective total of {params}.",
    "The combination of {params} must equal {param}.",
    "{param} equals the total sum of {params}.",
    "Summing {params} should yield {param}.",
    "The target of {param} is the sum of {params}.",
    "{param} should be achieved by adding {params}.",
    "The count of {param} equals the combined value of {params}.",
    "Make sure {param} is the total derived from {params}.",
    "{param} represents the sum total of {params}.",
    "The desired {param} is the total of {params}.",
    "Ensure {param} equals the addition of {params}.",
    "The goal is for {param} to be the sum of {params}.",
    "For a total of {param}, sum up {params}.",
    "The aim is to have {param} as the total from {params}.",
    "{param} should be the sum result of {params}.",
    "Achieve {param} by summing the values of {params}."
]
"""


"""
if "<=" == c.rel.operator:
      pass
elif ">=" == c.rel.operator:
      pass
elif "!=" == c.rel.operator:
      pass
elif "sigma_diff" == c.rel.operator:
      pass
elif "<" == c.rel.operator:
      pass
elif ">" == c.rel.operator:
      pass
elif "==" == c.rel.operator:
      pass
elif "||=val" == c.rel.operator:
      pass
elif "||!=val" == c.rel.operator:
      pass
elif "||>val" == c.rel.operator:
      pass
elif "||<val" == c.rel.operator:
      pass
elif "||>=val" == c.rel.operator:
      pass
elif "||<=val" == c.rel.operator:
      pass
elif "O_val_<=" == c.rel.operator:
      pass
elif "O_val_>=" == c.rel.operator:
      pass
elif "O_val_<" == c.rel.operator:
      pass
elif "O_val_>" == c.rel.operator:
      pass
elif "O_val_!=" == c.rel.operator:
      pass
elif "O_val_=" == c.rel.operator:
      pass
elif "||xy_zt" == c.rel.operator:
      pass
elif "||!=xy_zt" == c.rel.operator:
      pass
elif "Sum" == c.rel.operator:
      pass
else:
     print(f"Unsupported constraint: {c.rel.operator}")
     sys.exit()
"""


'\nif "<=" == c.rel.operator:\n      pass\nelif ">=" == c.rel.operator:\n      pass\nelif "!=" == c.rel.operator:\n      pass\nelif "sigma_diff" == c.rel.operator:\n      pass\nelif "<" == c.rel.operator:\n      pass\nelif ">" == c.rel.operator:\n      pass\nelif "==" == c.rel.operator:\n      pass\nelif "||=val" == c.rel.operator:\n      pass\nelif "||!=val" == c.rel.operator:\n      pass\nelif "||>val" == c.rel.operator:\n      pass\nelif "||<val" == c.rel.operator:\n      pass\nelif "||>=val" == c.rel.operator:\n      pass\nelif "||<=val" == c.rel.operator:\n      pass\nelif "O_val_<=" == c.rel.operator:\n      pass\nelif "O_val_>=" == c.rel.operator:\n      pass\nelif "O_val_<" == c.rel.operator:\n      pass\nelif "O_val_>" == c.rel.operator:\n      pass\nelif "O_val_!=" == c.rel.operator:\n      pass\nelif "O_val_=" == c.rel.operator:\n      pass\nelif "||xy_zt" == c.rel.operator:\n      pass\nelif "||!=xy_zt" == c.rel.operator:\n      pass\nelif "Sum" == c.rel.operator:\n      pa

In [37]:
"""
import nltk
import string
import json
import re
import random
nltk.download('punkt')

def format_template(template, **kwargs):
    try:
        formatted_template = template.format(**kwargs)
        return formatted_template
    except KeyError as e:
        print(f"Error: Missing parameter {e}")
        return None

def fill_template_sum(template, param, *params):
    params = ', '.join(map(str, params))  # Corrected: No '*' before 'params' in join
    return template.format(params=params, param=param)  # Corrected: Named placeholders in format


dict_constraints={
    0: {"constraint": "all_diff", "template":"templates_all_diff", "n_params":10},
    1: {"constraint": "p1<p2", "template":"templates_param1_strictely_lt_param2", "n_params":2},
    2: {"constraint": "p1>p2", "template":"templates_param1_strictely_gt_param2", "n_params":2},
    3: {"constraint": "|p1-p2|!=p3", "template":"abs_val_param1_minus_param_2_diff_param3", "n_params":3},
    4: {"constraint": "|p1-p2|=|p3-p4|", "template":"abs_val_p1_minus_p2_equal_abs_val_p3_minus_p4", "n_params":4},
    5: {"constraint": "|p1-p2|==p3", "template":"abs_val_p1_minus_p2_equal_p_3", "n_params":3},
    6: {"constraint": "p1+p2 != p3", "template":"templates_sum_x1_x2_diff_x3", "n_params":3},
    7: {"constraint": "p1=p2", "template":"templates_equal_constraint", "n_params":2},
    8: {"constraint": "p1!=p2", "template":"templates_different_than", "n_params":2},
    9: {"constraint": "p1<=p2", "template":"templates_less_than", "n_params":2},
    10: {"constraint": "p1 >= p2", "template":"templates_greater_than", "n_params":2},
    11: {"constraint": "|p1-p2|!=|p3-p4|", "template":"abs_val_p1_minus_p2_diff_abs_val_p3_minus_p4", "n_params":4},
    12: {"constraint": "sum", "template":"sum_params_equal_param", "n_params":10},
    13: {"constraint": "O_val=", "template":"templates_equal_constraint", "n_params":2},
    14: {"constraint": "O_val_!=", "template":"templates_different_than", "n_params":2},
    15: {"constraint": "||=val", "template":"abs_val_p1_minus_p2_equal_p_3", "n_params":3},
    16: {"constraint": "||!=val", "template":"abs_val_param1_minus_param_2_diff_param3", "n_params":3}
}

#I have to add data for O_val= and O_val_!= ||=val ||!=val

def generate_sythetic_sentence_for_constraint_i(i):

   params={}
   para=-1
"""
"""
  if c.rel.operator == "all_diff" :
    #print( " 1111 all_diff =", c.scope, c.rel.operator)
    params={"param"+str(k+1): f"X{c.scope[k]}" for k in range(len(c.scope))}
    selected_params = list( params.values())
    param_template = ", ".join(selected_params)
    formatted_template=template.format(params=param_template)

  elif c.rel.operator == "Sum":
    #print( " 2222 Sum =", c.scope, c.rel.operator)
    params={"param"+str(k+1): f"X{c.scope[k]}" for k in range(len(c.scope))}
    selected_params = list( params.values())
    para = c.rel.parameter
    formatted_template = fill_template_sum(template, para, *selected_params)

  else:
    params={"param"+str(k+1): f"X{c.scope[k]}" for k in range(len(c.scope))}
    if c.rel.operator in["||!=val", "||=val"]:
      params["param3"]=str(c.rel.parameter)
      #print( " 2222 ||!=val ||=val ", c.scope, c.rel.operator)
    elif c.rel.operator in[ "O_val_=", "O_val_!="]:
      params["param2"]=str(c.rel.parameter)
    else:
      params={"param"+str(k+1): f"X{c.scope[k]}" for k in range(len(c.scope))}
    formatted_template = format_template(template, **params)

    if i==0 :

      params={"param"+str(k+1): f"X{random_combination[k]}" for k in range(len(random_combination))}
      selected_params = list( params.values())
      template = random.choice(templates_all_diff)
      param_template = ", ".join(selected_params)
      formatted_template=template.format(params=param_template)

    elif i==12:

      num_params = random.randint(3, dict_constraints[i]["n_params"])
      values = list(range(100))
      random_combination = random.sample(values, num_params-1)
      params={"param"+str(k+1): f"X{random_combination[k]}" for k in range(len(random_combination))}
      selected_params = list( params.values())
      para = random.randint(0,100)
      template = random.choice(sum_params_equal_param)
      formatted_template = fill_template_sum(template, para, *selected_params)

    elif i==13:

      num_params = 2
      values = list(range(100))
      random_combination = random.sample(values, 1)
      params={"param"+str(k+1): f"X{random_combination[k]}" for k in range(len(random_combination))}
      para = random.randint(0,100)
      params["param"+str(2)]= str(para)
      template = random.choice(templates_equal_constraint)
      formatted_template = format_template(template, **params)

    elif i==14:

      num_params = 2
      values = list(range(100))
      random_combination = random.sample(values, num_params-1)
      params={"param"+str(k+1): f"X{random_combination[k]}" for k in range(len(random_combination))}
      para = random.randint(0,100)
      params["param"+str(2)]= str(para)
      template = random.choice(templates_different_than)
      formatted_template = format_template(template, **params)

    elif i==15:

      num_params = 3
      values = list(range(100))
      random_combination = random.sample(values, num_params-1)
      params={"param"+str(k+1): f"X{random_combination[k]}" for k in range(len(random_combination))}
      para = random.randint(0,100)
      params["param"+str(3)]= str(para)
      template = random.choice(abs_val_p1_minus_p2_equal_p_3)
      formatted_template = format_template(template, **params)

    elif i==16:

      num_params = 3
      values = list(range(100))
      random_combination = random.sample(values, num_params-1)
      params={"param"+str(k+1): f"X{random_combination[k]}" for k in range(len(random_combination))}
      para = random.randint(0,100)
      params["param"+str(3)]= str(para)
      template = random.choice(abs_val_param1_minus_param_2_diff_param3)
      formatted_template = format_template(template, **params)

    else:

      num_params = dict_constraints[i]["n_params"]
      for k in range(num_params):
        params["param"+str(k+1)]=f"X{random.randint(1, 100)}"
      template = dict_constraints[i]["template"]
      template= random.choice(eval(template))
      formatted_template = format_template(template, **params)

    return formatted_template,params,str(para)


pattern = r'param(\d+)'
n_constraints= 17 #11
n_samples_by_constraint = 500
filename = 'synthetic_data_221123.txt'
with open(filename, 'w') as file:
  for i in range(n_constraints):
  #for i in [16]:
    n_samples=0
    while n_samples < n_samples_by_constraint :
      sentence,my_params,para=generate_sythetic_sentence_for_constraint_i(i)
      n_samples=n_samples+1
      annotations = []
        # Tokenizing using NLTK's word_tokenize
      #print(" sentence..",sentence)
      words = nltk.word_tokenize(sentence)
      # Removing punctuation
      tokens = [word for word in words if word not in string.punctuation]
      labels = {v: k for k, v in my_params.items()}
      #print("labels", labels)
      vars  = list( my_params.values())
      vars=[str(m) for m in vars]
      #print("vars", vars)
      #print("tokens",tokens)
      for token in tokens:
          print(token)
          if token in vars:
            match = re.search(pattern, labels[token])
            annotations.append( int(match.group(1)) )
          elif token == para:
            #print(" token not in vars")
            annotations.append( 10 )
          else:
                annotations.append(0)
      #print(sentence, my_params, dict_constraints[i]["constraint"])
      line_1={}
      line_1["text"]= ' '.join(tokens)
      line_1["labels"]= f"{annotations}"
      line_1["task_name"]= "token_classification"
      #{"text": "This is a positive sentence.", "labels": [1], "task_name": "sequence_classification"},#
      #print("line ",line_1)
      file.write(json.dumps(line_1) + '\n')

      line_2={}
      line_2["text"]= ' '.join(tokens)
      line_2["labels"]= f"{[i]}"
      line_2["task_name"]= "sequence_classification"
      #{"text": "This is a positive sentence.", "labels": [1], "task_name": "sequence_classification"},#
      #print("line ",line_2)
      file.write(json.dumps(line_2) + '\n')
      n_samples= n_samples+1

"""

'\n  if c.rel.operator == "all_diff" :\n    #print( " 1111 all_diff =", c.scope, c.rel.operator)\n    params={"param"+str(k+1): f"X{c.scope[k]}" for k in range(len(c.scope))}\n    selected_params = list( params.values())\n    param_template = ", ".join(selected_params)\n    formatted_template=template.format(params=param_template)\n\n  elif c.rel.operator == "Sum":\n    #print( " 2222 Sum =", c.scope, c.rel.operator)\n    params={"param"+str(k+1): f"X{c.scope[k]}" for k in range(len(c.scope))}\n    selected_params = list( params.values())\n    para = c.rel.parameter\n    formatted_template = fill_template_sum(template, para, *selected_params)\n\n  else:\n    params={"param"+str(k+1): f"X{c.scope[k]}" for k in range(len(c.scope))}\n    if c.rel.operator in["||!=val", "||=val"]:\n      params["param3"]=str(c.rel.parameter)\n      #print( " 2222 ||!=val ||=val ", c.scope, c.rel.operator)\n    elif c.rel.operator in[ "O_val_=", "O_val_!="]:\n      params["param2"]=str(c.rel.parameter)\n   

In [38]:

############## TO BE REMOVED
"""
rel1= Relation('<=', 2, True)
rel2= Relation('!=',2, False)
rel3= Relation('sigma_diff',3, False )
rel4= Relation('<', 2, True )
rel5= Relation('==',2, False )
rel6= Relation('>',2, False )
rel7= Relation('>=', 2, True )
rel8=Relation('||=val',2,False )
rel9=Relation('||!=val',2,False )
rel10= Relation('||>val',1,False )
rel11= Relation('O_val_<=',1,False )
rel12= Relation('O_val_>=',1,False )
rel13= Relation('O_val_<',1,False )
rel14= Relation('O_val_>',1,False )
rel15= Relation('O_val_!=',1,False )
rel16= Relation('O_val_=',1,False )
rel17=Relation('||xy_zt',4,False )
rel18=Relation('||!=xy_zt',4,False )
rel19= Relation('Sum',10,False)
rel20= Relation('all_diff',10,False)



# add constraint constraints to the solver

dict_templates={
  "all_diff" :"templates_all_diff",
  "<":"templates_param1_strictely_lt_param2" ,
  ">":"templates_param1_strictely_gt_param2" ,
  "||!=val":"abs_val_param1_minus_param_2_diff_param3", #?????
  "||xy_zt" :"abs_val_p1_minus_p2_equal_abs_val_p3_minus_p4",
  "sigma_diff":"templates_sum_x1_x2_diff_x3",
  "==":"templates_equal_constraint" ,
  "!=":"templates_different_than" ,
  "<=":"templates_less_than" ,
  ">=":"templates_greater_than" ,
  "||!=xy_zt":"abs_val_p1_minus_p2_diff_abs_val_p3_minus_p4" ,
  "||=val":"abs_val_p1_minus_p2_equal_p_3" ,
  "Sum" :"sum_params_equal_param"
}

def format_template(template, **kwargs):
    try:
        formatted_template = template.format(**kwargs)
        return formatted_template
    except KeyError as e:
        print(f"Error: Missing parameter {e}")
        return None

def fill_template_sum(template, param, *params):
    params = ', '.join(map(str, params))  # Corrected: No '*' before 'params' in join
    #print("i m here", params, param)
    return template.format(params=params, param=param)  # Corrected: Named placeholders in format


#template=[c.rel.operator]


def generateSentenceForConstraint(c):
  #print("####################### call to generate sentence")

  params={}
  para=-1
  template=""
  if c.rel.operator == "O_val_=" :
    template= dict_templates["=="]

  elif c.rel.operator == "O_val_=!" :
    template= dict_templates["!="]

  else:
    template = dict_templates[c.rel.operator]


  template= random.choice(eval(template))


  if c.rel.operator == "all_diff" :
    #print( " 1111 all_diff =", c.scope, c.rel.operator)
    params={"param"+str(k+1): f"X{c.scope[k]}" for k in range(len(c.scope))}
    selected_params = list( params.values())
    param_template = ", ".join(selected_params)
    formatted_template=template.format(params=param_template)

  elif c.rel.operator == "Sum":
    #print( " 2222 Sum =", c.scope, c.rel.operator)
    params={"param"+str(k+1): f"X{c.scope[k]}" for k in range(len(c.scope))}
    selected_params = list( params.values())
    para = c.rel.parameter
    formatted_template = fill_template_sum(template, para, *selected_params)

  else:
    params={"param"+str(k+1): f"X{c.scope[k]}" for k in range(len(c.scope))}
    if c.rel.operator in["||!=val", "||=val"]:
      params["param3"]=str(c.rel.parameter)
      #print( " 2222 ||!=val ||=val ", c.scope, c.rel.operator)
    elif c.rel.operator in[ "O_val_=", "O_val_!="]:
      params["param2"]=str(c.rel.parameter)
    else:
      params={"param"+str(k+1): f"X{c.scope[k]}" for k in range(len(c.scope))}
    formatted_template = format_template(template, **params)

  return formatted_template
"""


'\nrel1= Relation(\'<=\', 2, True)\nrel2= Relation(\'!=\',2, False)\nrel3= Relation(\'sigma_diff\',3, False )\nrel4= Relation(\'<\', 2, True )\nrel5= Relation(\'==\',2, False )\nrel6= Relation(\'>\',2, False )\nrel7= Relation(\'>=\', 2, True )\nrel8=Relation(\'||=val\',2,False )\nrel9=Relation(\'||!=val\',2,False )\nrel10= Relation(\'||>val\',1,False )\nrel11= Relation(\'O_val_<=\',1,False )\nrel12= Relation(\'O_val_>=\',1,False )\nrel13= Relation(\'O_val_<\',1,False )\nrel14= Relation(\'O_val_>\',1,False )\nrel15= Relation(\'O_val_!=\',1,False )\nrel16= Relation(\'O_val_=\',1,False )\nrel17=Relation(\'||xy_zt\',4,False )\nrel18=Relation(\'||!=xy_zt\',4,False )\nrel19= Relation(\'Sum\',10,False)\nrel20= Relation(\'all_diff\',10,False)\n\n\n\n# add constraint constraints to the solver\n\ndict_templates={\n  "all_diff" :"templates_all_diff",\n  "<":"templates_param1_strictely_lt_param2" ,\n  ">":"templates_param1_strictely_gt_param2" ,\n  "||!=val":"abs_val_param1_minus_param_2_diff_para

In [39]:

"""
for clue in vertical_clues:
  print(" printing vertical clues ...")
  val = clue
  list_vars = [t[0]*n+t[1] for t in  vertical_clues[clue]]
  print(clue,  list_vars)
  constraints.append(Constraint(list_vars, Relation('Sum',10,False,val)))
  constraints.append(Constraint(list_vars, r39))

print("le nombre des contraintes est ", len(constraints))
target_net= Target_net(constraints)
"""

"""
constraints=[]
for ligne in range(n):
  for i in all_diff_on_lines[ligne] :
    for j in all_diff_on_lines[ligne]:
      if j>i:
        constraints.append( Constraint([i,j],r6))

for colone in range(n):
  for i in all_diff_on_cols[colone] :
    for j in all_diff_on_cols[colone]:
      if j>i:
        constraints.append( Constraint([i,j],r6))

c1= Constraint([0, 1, 2, 3, 4],r19)
c2= Constraint([6, 7, 8, 9, 10, 11],r20)
cs = [c1,c2]

constraints.extend(cs)
"""

"""
all_diff_on_lines={}
for i in range(n):
  all_diff_on_lines[i]=[i*n+j for j in range(n) if grid[i][j]==0]

all_diff_on_cols={}
for j in range(n):
  all_diff_on_cols[j]=[i*n+j for i in range(n) if grid[i][j]==0]

print(all_diff_on_lines)
print(all_diff_on_cols)
"""

# all diff constraints
"""
for i in range(n) :
  constraints.append( Constraint(all_diff_on_lines[i],r39))
  constraints.append( Constraint(all_diff_on_cols[i],r39))
"""

'\nfor i in range(n) :\n  constraints.append( Constraint(all_diff_on_lines[i],r39))\n  constraints.append( Constraint(all_diff_on_cols[i],r39))\n'